# **Part C. Consumption level for the selected neighbourhood**

## C1. Import libaries

In [ ]:
#!conda install pandas --yes

In [ ]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests 
#from bs4 import BeautifulSoup
#import html5lib

In [ ]:
import itertools

import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter

import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline
# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y
import seaborn as sns

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.11.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:
!pip install geocoder #get latitude and longtitude if know address/postalcode

!conda install -c districtdatalabs yellowbrick --yes #for kmean

!pip install lxml

!pip install BeautifulSoup4
from bs4 import BeautifulSoup

In [ ]:
!pip install geopandas

## Import SQL SERVER

Identify the database connection credentials

In [ ]:
import ibm_db

In [ ]:
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"
dsn_hostname ="dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net"
dsn_port = "50000" 
dsn_protocol = "TCPIP"
dsn_uid ="gln55437"
dsn_pwd ="v6jk^s58hvsjd4lj"     

Create the database connection

In [ ]:
#Create database connection
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

In [ ]:
import pandas
import ibm_db_dbi

In [ ]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

## C2. Import data

### C2.1 Core/Mature/Established/Developing Neighbourhood with ≥10 Millennials per net residential hectare

In [ ]:
%store -r dtopNei
dtopNei.tail()

In [ ]:
%store -r topNei_Mdensity
topNei_Mdensity.tail()

In [ ]:
%store -r dclassMED
dclassMED.tail()

In [ ]:
%store -r dN_2016n2

## C3. Top neighbourhoods' Dwelling status
Reference:
 - https://data.edmonton.ca/Census/2019-Census-Dwelling-Unit-Status-Neighbourhood-War/522q-3877 2019 Census - Dwelling Unit Status (Neighbourhood/Ward)

In [ ]:
d2019dwellsta = pd.read_csv("https://data.edmonton.ca/api/views/522q-3877/rows.csv?accessType=DOWNLOAD")
d2019dwellsta.head()

In [ ]:
d2019dwellsta.rename(columns={'Neighbourhood Name':'Neighbourhood'}, inplace=True)
d2019DS = pd.merge(dtopNei,
                d2019dwellsta[['Neighbourhood','Occupied','Unoccupied','Refuses to Answer']],
                 on='Neighbourhood')
d2019DS['Total']=d2019DS['Occupied']+d2019DS['Unoccupied']+d2019DS['Refuses to Answer']
d2019DS['Percentage Occupied (%)']=d2019DS['Occupied']/d2019DS['Total']*100
d2019DS['Percentage Unoccupied (%)']=d2019DS['Unoccupied']/d2019DS['Total']*100
d2019DS['Percentage Refuses to Answer (%)']=d2019DS['Refuses to Answer']/d2019DS['Total']*100
d2019DS.sort_values(['Percentage Occupied (%)'], inplace=True ,ascending=True)
d2019DS.reset_index(drop=True, inplace=True)
d2019DS.tail()

### Heat map

Get the geographical coordinates for each borough in Edmonton

In [ ]:
address = 'Edmonton, AB'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Edmonton are {}, {}.'.format(latitude, longitude))

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisA.tail()

In [ ]:
NeisA.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisA,
                dtopNei,
                 on='Neighbourhood')
NeiAmerge = pd.merge(NeiAmerge,
                d2019DS[['Neighbourhood','Percentage Occupied (%)']],
                 on='Neighbourhood')
NeiAmerge.rename(columns={'Neighbourhood':'name'}, inplace=True)
NeiAmerge['centroid_long']=NeiAmerge.centroid.x
NeiAmerge['centroid_lag']=NeiAmerge.centroid.y
NeiAmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09#PuOr
colormap = cm.linear.Oranges_03.scale(70,100).to_step(3) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron') #, tiles = 'Stamen Toner')
colormap.caption = "Percentage Occupied (%)"

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']["Percentage Occupied (%)"]), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name',"Percentage Occupied (%)"],
        aliases=['name',"Percentage Occupied (%)"],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

colormap.add_to(edmA_map)

from branca.element import Template, MacroElement
class DivIcon(MacroElement):
    def __init__(self, html='', size=(5,5), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)
for i in range(len(NeiAmerge)):
    lag=NeiAmerge.iloc[i][6]
    log=NeiAmerge.iloc[i][5]
    text=NeiAmerge.iloc[i][2]
    folium.map.Marker(
        [lag+0.003, log-0.005],
        icon=DivIcon(
            size=(5,5),
            anchor=(0,0),
            html=text,
            style="""
            font-size:8px;
            background-color: transparent;
            border-color: transparent;
            text-align: center;
            """
        )
    ).add_to(edmA_map)
    
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body> 

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Numbers of people in age range 25-39 per net residential hectareper in mature and central core area</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#fee0b6;opacity:0.7;'></span><80%</li>
    <li><span style='background:#f1a340;opacity:0.7;'></span>80%-90%</li>
    <li><span style='background:#b35806;opacity:0.7;'></span>>90%</li>


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }  
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edmA_map.get_root().add_child(macro)
   
edmA_map

### Bar chart

#check whether respondse% over 25% of the total population
check = pd.merge(d2019DS,
                dN_2016n2[['Neighbourhood','Total']],
                 on='Neighbourhood')
check['% respond']=(check['Occupied']+check['Unoccupied'])/check['Total_y']


In [ ]:
D2019Ds=d2019DS.copy()
D2019Ds.drop(['Total','Occupied','Unoccupied','Refuses to Answer'],axis=1, inplace=True)
D2019Ds.set_index('Neighbourhood', inplace=True)
D2019Ds.tail()

In [ ]:
color_list = ['plum','khaki','lemonchiffon'] 

ax = D2019Ds.plot.barh(stacked=True,figsize = (20, 20), width = 0.8, color = color_list, align='center', edgecolor = None);



ax.set_title("2019 Dwelling Status", fontsize = '16')#use font size 16 for the title
ax.legend(fontsize='14',facecolor='white')#use font size 14 for legend
ax.tick_params(axis='x', labelsize='14')#use font size 14 for axis-lable
ax.set_xlabel('Percentage') 
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(str(width.round(1))+'%', xy=(left+width/2, bottom+height/2), 
                ha='center', va='center')
    
#ax.set_xlim(0, 35)
    
# remove the left, top, and right borders; make sure the color of x axix labels is black
ax.set_facecolor('xkcd:white') # change background to white  #ax.set_facecolor('xkcd:white')
#ax.set_xticklabels([]) # turn off y ticks
ax.tick_params(left=False)
ax.axvline(0, color='black') # draw an x axix line 
ax.tick_params(axis='x', colors='black') # make sure the color of x axix labels is black    

## C4. Import Dwelling ownnership status     
**(Usually, people own a dwelling spend more on food from restaurant)**    
Reference:
 - https://data.edmonton.ca/Census/2016-Census-Dwelling-Unit-by-Ownership-Neighbourho/d2xp-ctch 2016 Census - Dwelling Unit by Ownership (Neighbourhood/Ward)

In [ ]:
d_ownership = pd.read_csv("https://data.edmonton.ca/api/views/d2xp-ctch/rows.csv?accessType=DOWNLOAD")
d_ownership.tail()

### C4.1 Clean data

In [ ]:
d_ownership.rename(columns={'Neighbourhood Name':'Neighbourhood'}, inplace=True)
d_ownership['Neighbourhood']=d_ownership['Neighbourhood'].str.title()
ownership2016 = pd.merge(dtopNei,
                d_ownership[['Neighbourhood','Owned','Rented','No Response']],
                 on='Neighbourhood')
ownership2016.tail()

In [ ]:
dm1=ownership2016.copy()
dm2=dtopNei.copy()
dm1.drop(['Owned','Rented','No Response'],axis=1, inplace=True)

#dm2.reset_index(drop=True, inplace=True)

dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

In [ ]:
dmiss['Owned']=[1231,1216,321,227]
dmiss['Rented']=[346,587,73,2471]
dmiss['No Response']=[341,431,119,295]
dmiss

In [ ]:
ownership2016= ownership2016.append(dmiss, ignore_index=True)

ownership2016['Total']=ownership2016['Owned']+ownership2016['Rented']+ownership2016['No Response']
ownership2016['Owned (%)']=ownership2016['Owned']/ownership2016['Total']*100
ownership2016['Rented (%)']=ownership2016['Rented']/ownership2016['Total']*100
ownership2016['No Response (%)']=ownership2016['No Response']/ownership2016['Total']*100
ownership2016.sort_values(['Owned (%)'], inplace=True ,ascending=True)
ownership2016.reset_index(drop=True, inplace=True)

ownership2016.tail()

### Heat map

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisA.tail()

In [ ]:
NeisA.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisA,
                dtopNei,
                 on='Neighbourhood')
NeiAmerge = pd.merge(NeiAmerge,
                ownership2016[['Neighbourhood','Owned (%)']],
                 on='Neighbourhood')
NeiAmerge.rename(columns={'Neighbourhood':'name'}, inplace=True)
NeiAmerge['centroid_long']=NeiAmerge.centroid.x
NeiAmerge['centroid_lag']=NeiAmerge.centroid.y
NeiAmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09#PuOr
colormap = cm.linear.PiYG_06.scale(0,90).to_step(6) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron') #, tiles = 'Stamen Toner')
colormap.caption = "Dwelling Owned (%)"

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']["Owned (%)"]), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name',"Owned (%)"],
        aliases=['name',"Owned (%)"],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

colormap.add_to(edmA_map)

from branca.element import Template, MacroElement
class DivIcon(MacroElement):
    def __init__(self, html='', size=(5,5), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)
for i in range(len(NeiAmerge)):
    lag=NeiAmerge.iloc[i][6]
    log=NeiAmerge.iloc[i][5]
    text=NeiAmerge.iloc[i][2]
    folium.map.Marker(
        [lag+0.003, log-0.005],
        icon=DivIcon(
            size=(5,5),
            anchor=(0,0),
            html=text,
            style="""
            font-size:8px;
            background-color: transparent;
            border-color: transparent;
            text-align: center;
            """
        )
    ).add_to(edmA_map)
#edmA_map

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Dwelling Owned (%)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#c51b7d;opacity:0.7;'></span><15%</li>
    <li><span style='background:#e9a3c9;opacity:0.7;'></span>15% to 30%</li>
    <li><span style='background:#fde0ef;opacity:0.7;'></span>30% to 45%</li>
    <li><span style='background:#e6f5d0;opacity:0.7;'></span>45% to 60%</li>
    <li><span style='background:#a1d76a;opacity:0.7;'></span>60% to 75%</li>
    <li><span style='background:#4d9221;opacity:0.7;'></span>>75%</li>


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edmA_map.get_root().add_child(macro)

edmA_map

### Bar chart

In [ ]:
#check whether respondse% over 25% of the total population
check = pd.merge(ownership2016,
                dN_2016n2[['Neighbourhood','Total']],
                 on='Neighbourhood')
check['% respond']=(check['Owned']+check['Rented'])/check['Total_y']


In [ ]:
D2016DO=ownership2016.copy()
D2016DO.drop(['Total','Owned','Rented','No Response'],axis=1, inplace=True)
D2016DO.set_index('Neighbourhood', inplace=True)
D2016DO.tail()

In [ ]:
color_list = ['plum','khaki','lemonchiffon'] 

ax = D2016DO.plot.barh(stacked=True,figsize = (20, 20), width = 0.8, color = color_list, align='center', edgecolor = None);



ax.set_title("2019 Dwelling Ownership Status", fontsize = 16)#use font size 16 for the title
ax.legend(fontsize='14',facecolor='white')#use font size 14 for legend
ax.tick_params(axis='x', labelsize=14)#use font size 14 for axis-lable
ax.set_xlabel('Percentage') 
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(str(width.round(1))+'%', xy=(left+width/2, bottom+height/2), 
                ha='center', va='center')
    
#ax.set_xlim(0, 35)
    
# remove the left, top, and right borders; make sure the color of x axix labels is black
ax.set_facecolor('xkcd:white') # change background to white  #ax.set_facecolor('xkcd:white')
#ax.set_xticklabels([]) # turn off y ticks
ax.tick_params(left=False)
ax.axvline(0, color='black') # draw an x axix line 
ax.tick_params(axis='x', colors='black') # make sure the color of x axix labels is black    

### C4.2 Dwelling ownership status_mortgage

###### M1c.1 Connect to the database from SQL

Import data from SQL SERVER

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from SHELTER_COST"

#retrieve the query results into a pandas dataframe
f_morg = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
f_morg.head(6)

### Clean data (Federal census)

In [ ]:
f_morg.rename(columns={'Name':'Neighbourhood','Owner_Measure_Value_PT':'With/Without Mortgage (%)'}, inplace=True)
f_morg['Neighbourhood']=f_morg['Neighbourhood'].str.title()
f_morg.drop(['COE_OwnerMeaure_Value_PT','1'],axis=1, inplace=True)
f_morg['Neighbourhood']=f_morg['Neighbourhood'].replace({'Macewan':'MacEwan','Central Mcdougall': 'Central McDougall','Mcconachie Area':'McConachie Area'})
f_morg.sort_values(['Neighbourhood'], inplace=True ,ascending=True)


morg2016 = pd.merge(f_morg,
                      dtopNei[['Neighbourhood']],
                      on='Neighbourhood')
morg2016.head(10)

In [ ]:
dm1=morg2016.copy()
dm2=dtopNei.copy()
dm1=dm1.groupby('Neighbourhood',axis=0)['With/Without Mortgage (%)'].sum().reset_index()
dm1=dm1[['Neighbourhood']]
dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

In [ ]:
S_morg2016=morg2016.copy()
S_morg2016=S_morg2016.pivot(index='Neighbourhood',columns='Owner_Measure_Type',values='With/Without Mortgage (%)')
S_morg2016=S_morg2016.astype(float)
S_morg2016.sort_values(['% of HH with a mortgage'], inplace=True ,ascending=True)

S_morg2016.head()

In [ ]:
a=S_morg2016.copy()
a=a['% of HH with a mortgage'].reset_index()
b=S_morg2016.copy()
b=b['% of HH without a mortgage'].reset_index()

morg2016 = pd.merge(a,
                b,
                 on='Neighbourhood')

morg2016.set_index('Neighbourhood', inplace=True)
morg2016.tail()

In [ ]:
color_list = ['plum','khaki','lemonchiffon'] 

ax = morg2016.plot.barh(stacked=True,figsize = (20, 20), width = 0.8, color = color_list, align='center', edgecolor = None);



ax.set_title("2019 Education Status", fontsize = 16)#use font size 16 for the title
ax.legend(fontsize='14',facecolor='white',loc="lower right")#use font size 14 for legend
ax.tick_params(axis='x', labelsize=14)#use font size 14 for axis-lable
ax.set_xlabel('Percentage') 
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(str(width.round(1))+'%', xy=(left+width/2, bottom+height/2), 
                ha='center', va='center')
    
#ax.set_xlim(0, 35)
    
# remove the left, top, and right borders; make sure the color of x axix labels is black
ax.set_facecolor('xkcd:white') # change background to white  #ax.set_facecolor('xkcd:white')
#ax.set_xticklabels([]) # turn off y ticks
ax.tick_params(left=False)
ax.axvline(0, color='black') # draw an x axix line 
ax.tick_params(axis='x', colors='black') # make sure the color of x axix labels is black    


## C5. Import Marital status    
**(Usually, for one-person households, the average person food Expenditures tend to the most compared to other types households)**    
Reference:    
 - https://data.edmonton.ca/Census/2016-Census-Population-by-Marital-Status-Neighbour/fcvm-p6fa 2016 Census - Population by Marital Status (Neighbourhood/Ward)

In [ ]:
d_marital = pd.read_csv("https://data.edmonton.ca/api/views/fcvm-p6fa/rows.csv?accessType=DOWNLOAD")
d_marital.tail()

### C5.1 Clean data

In [ ]:
d_marital.rename(columns={'Neighbourhood Name':'Neighbourhood'}, inplace=True)
d_marital['Neighbourhood']=d_marital['Neighbourhood'].str.title()
d_marital.drop(['Ward','Neighbourhood Number'],axis=1, inplace=True)

d_marital['Neighbourhood']=d_marital['Neighbourhood'].replace({'Mcconachie Area':'McConachie Area','Macewan':'MacEwan','Glenridding Area':'Glenridding Heights','Central Mcdougall': 'Central McDougall'})
d_marital.sort_values(['Neighbourhood'], inplace=True ,ascending=True)

marital2016 = pd.merge(d_marital,
                dtopNei[['Neighbourhood']],
                 on='Neighbourhood')
marital2016.tail()

In [ ]:
dm1=marital2016.copy()
dm2=dtopNei.copy()
dm1=dm1[['Neighbourhood']]


dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

In [ ]:
marital2016.set_index('Neighbourhood', inplace=True)
marital2016.tail()

In [ ]:
#check whether respondse% over 25% of the total population
check = pd.merge(marital2016,
                dN_2016n2[['Neighbourhood','Total']],
                 on='Neighbourhood')
check['% respond']=(check['Married']+check['Common Law']+check['Separated/Divorced']+check['Never Married']+check['Widowed'])/check['Total']


In [ ]:
marital2016c=marital2016.copy()
marital2016c.drop(['No Response'],axis=1, inplace=True)
cols_to_sum = marital2016c.columns[ 0: marital2016.shape[1]]
marital2016c['Total']=marital2016c[cols_to_sum].sum(axis=1)

marital2016c['Married (%)']=marital2016c['Married']/marital2016c['Total']*100
marital2016c['Common Law (%)']=marital2016c['Common Law']/marital2016c['Total']*100

marital2016c['Separated/Divorced (%)']=marital2016c['Separated/Divorced']/marital2016c['Total']*100
marital2016c['Widowed (%)']=marital2016c['Widowed']/marital2016c['Total']*100

marital2016c['1 person household (%)']=marital2016c['Never Married']/marital2016c['Total']*100
marital2016c['≥2 person household (%)']=marital2016c['Married (%)']+marital2016c['Common Law (%)']
marital2016c['other (%)']=marital2016c['Separated/Divorced (%)']+marital2016c['Widowed (%)']

marital2016c.drop(['Married','Common Law','Separated/Divorced','Widowed','Total','Never Married','Married (%)','Common Law (%)','Separated/Divorced (%)','Widowed (%)'],axis=1, inplace=True)
marital2016c.sort_values(['1 person household (%)'], inplace=True ,ascending=True)
marital2016c.tail()

### Heat map

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisA.tail()

In [ ]:
NeisA.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisA,
                dtopNei,
                 on='Neighbourhood')
NeiAmerge = pd.merge(NeiAmerge,
                marital2016c,
                 on='Neighbourhood')
NeiAmerge.rename(columns={'Neighbourhood':'name'}, inplace=True)
NeiAmerge['centroid_long']=NeiAmerge.centroid.x
NeiAmerge['centroid_lag']=NeiAmerge.centroid.y
NeiAmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09#PuOr
colormap = cm.linear.RdBu_06.scale(0,60).to_step(6) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron') #, tiles = 'Stamen Toner')
colormap.caption = "1 person household (%)"

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']["1 person household (%)"]), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name',"1 person household (%)"],
        aliases=['name',"1 person household (%)"],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

colormap.add_to(edmA_map)

from branca.element import Template, MacroElement
class DivIcon(MacroElement):
    def __init__(self, html='', size=(5,5), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)
for i in range(len(NeiAmerge)):
    lag=NeiAmerge.iloc[i][8]
    log=NeiAmerge.iloc[i][7]
    text=NeiAmerge.iloc[i][2]
    folium.map.Marker(
        [lag+0.003, log-0.005],
        icon=DivIcon(
            size=(5,5),
            anchor=(0,0),
            html=text,
            style="""
            font-size:8px;
            background-color: transparent;
            border-color: transparent;
            text-align: center;
            """
        )
    ).add_to(edmA_map)
#edmA_map

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>1 person household (%)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#b2182b;opacity:0.7;'></span><10%</li>
    <li><span style='background:#ef8a62;opacity:0.7;'></span>10% to 20%</li>
    <li><span style='background:#fddbc7;opacity:0.7;'></span>20% to 30%</li>
    <li><span style='background:#d1e5f0;opacity:0.7;'></span>30% to 40%</li>
    <li><span style='background:#67a9cf;opacity:0.7;'></span>50% to 60%</li>
    <li><span style='background:#2166ac;opacity:0.7;'></span>>60%</li>


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edmA_map.get_root().add_child(macro)

edmA_map

### Bar chart

In [ ]:
color_list = ['plum','khaki','lemonchiffon'] 

ax = marital2016c.plot.barh(stacked=True,figsize = (20, 20), width = 0.8, color = color_list, align='center', edgecolor = None);



ax.set_title("2019 Marital Status", fontsize = 16)#use font size 16 for the title
ax.legend(fontsize='14',facecolor='white')#use font size 14 for legend
ax.tick_params(axis='x', labelsize=14)#use font size 14 for axis-lable
ax.set_xlabel('Percentage') 
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(str(width.round(1))+'%', xy=(left+width/2, bottom+height/2), 
                ha='center', va='center')
    
#ax.set_xlim(0, 35)
    
# remove the left, top, and right borders; make sure the color of x axix labels is black
ax.set_facecolor('xkcd:white') # change background to white  #ax.set_facecolor('xkcd:white')
#ax.set_xticklabels([]) # turn off y ticks
ax.tick_params(left=False)
ax.axvline(0, color='black') # draw an x axix line 
ax.tick_params(axis='x', colors='black') # make sure the color of x axix labels is black    

## C6. Family with/without children
Reference：
 - https://public.tableau.com/profile/city.of.edmonton#!/vizhome/NeighbourhoodProfiles_FederalCensus2016/FamilyandLanguage
1. Download Tableau from the website
2. Access data by clicking 'data' from tableau and explore all into csv
3. upload to IBM_Cloud→Db2-vs→Manage→Open console→Load data (under GLN55437)

### Import data

###### M1c.1 Connect to the database from SQL

Import data from SQL SERVER

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from FAMILY_COUPLE"

#retrieve the query results into a pandas dataframe
f_couple = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
f_couple.head()

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from FAMILY_LONEPARENT"

#retrieve the query results into a pandas dataframe
f_lone = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
f_lone.head()

### C6.1 Clean data

In [ ]:
f_couple.rename(columns={'NAME':'Neighbourhood','Couple_Children_POP':'Family Number','Couple_Children_Group':'Children Group'}, inplace=True)
f_couple['Neighbourhood']=f_couple['Neighbourhood'].str.title()
f_couple.drop(['Nbhd_Num','Couple_Children_Percentage'],axis=1, inplace=True)
f_couple['Neighbourhood']=f_couple['Neighbourhood'].replace({'Macewan':'MacEwan','Central Mcdougall': 'Central McDougall','Mcconachie Area':'McConachie Area'})
f_couple.sort_values(['Neighbourhood'], inplace=True ,ascending=True)


couple2016 = pd.merge(f_couple,
                      dtopNei[['Neighbourhood']],
                      on='Neighbourhood')
couple2016.tail()

In [ ]:
dm1=couple2016.copy()
dm2=dtopNei.copy()
dm1=dm1.groupby('Neighbourhood',axis=0)['Family Number'].sum().reset_index()
dm1=dm1[['Neighbourhood']]
dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

In [ ]:
f_lone.rename(columns={'NAME':'Neighbourhood','Lone_Parent_Children_POP':'Family Number','Lone_Parent_Children':'Children Group'}, inplace=True)
f_lone['Neighbourhood']=f_lone['Neighbourhood'].str.title()
f_lone.drop(['Nbhd_Num','Lone_Parent_Children_Percentage'],axis=1, inplace=True)
f_lone['Neighbourhood']=f_lone['Neighbourhood'].replace({'Macewan':'MacEwan','Glenridding Area':'Glenridding Heights','Central Mcdougall': 'Central McDougall','Mcconachie Area':'McConachie Area'})
f_lone.sort_values(['Neighbourhood'], inplace=True ,ascending=True)


lone2016 = pd.merge(f_lone,
                      dtopNei[['Neighbourhood']],
                      on='Neighbourhood')
lone2016.tail()

In [ ]:
fami2016=couple2016.copy()
fami2016= fami2016.append(lone2016, ignore_index=True)
fami2016.sort_values(['Neighbourhood'], inplace=True ,ascending=True)
fami2016.reset_index(drop=True, inplace=True)
fami2016.head(10)

In [ ]:
fami2016['Family Number']=fami2016['Family Number'].astype(float)
fami2016 = fami2016.groupby(['Children Group','Neighbourhood'],axis=0)['Family Number'].sum().reset_index()
fami2016.sort_values(['Neighbourhood'], inplace=True ,ascending=True)
fami2016.reset_index(drop=True, inplace=True)
fami2016.head(10)

In [ ]:
S_fami2016=fami2016.copy()
S_fami2016=S_fami2016.pivot(index='Neighbourhood',columns='Children Group',values='Family Number')
S_fami2016['Total']=S_fami2016.sum(axis=1)
S_fami2016['Without children (%)']=(S_fami2016['Without children']/S_fami2016['Total']*100).round(2)
S_fami2016['With children (%)']=((S_fami2016['One child']+S_fami2016['Two children']+S_fami2016['Three or more children'])/S_fami2016['Total']*100).round(2)
S_fami2016.sort_values(['Without children (%)'], inplace=True ,ascending=True)
S_fami2016.head(10)

In [ ]:

a=S_fami2016.copy()
a=a['Without children (%)'].reset_index()
b=S_fami2016.copy()
b=b['With children (%)'].reset_index()
family2016 = pd.merge(a,
                b,
                 on='Neighbourhood')
family2016.set_index('Neighbourhood', inplace=True)
family2016.tail()

### Heat map

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisA.tail()

In [ ]:
NeisA.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisA,
                dtopNei,
                 on='Neighbourhood')
NeiAmerge = pd.merge(NeiAmerge,
                family2016,
                 on='Neighbourhood')
NeiAmerge.rename(columns={'Neighbourhood':'name'}, inplace=True)
NeiAmerge['centroid_long']=NeiAmerge.centroid.x
NeiAmerge['centroid_lag']=NeiAmerge.centroid.y
NeiAmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09#PuOr
colormap = cm.linear.PuOr_06.scale(-30,90).to_step(8) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron') #, tiles = 'Stamen Toner')
colormap.caption = "Without children (%)"

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']["Without children (%)"]), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name',"Without children (%)"],
        aliases=['name',"Without children (%)"],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

colormap.add_to(edmA_map)

from branca.element import Template, MacroElement
class DivIcon(MacroElement):
    def __init__(self, html='', size=(5,5), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)
for i in range(len(NeiAmerge)):
    lag=NeiAmerge.iloc[i][7]
    log=NeiAmerge.iloc[i][6]
    text=NeiAmerge.iloc[i][2]
    folium.map.Marker(
        [lag+0.003, log-0.005],
        icon=DivIcon(
            size=(5,5),
            anchor=(0,0),
            html=text,
            style="""
            font-size:8px;
            background-color: transparent;
            border-color: transparent;
            text-align: center;
            """
        )
    ).add_to(edmA_map)
#edmA_map

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Without children (%)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#f1a340;opacity:0.7;'></span><15%</li>
    <li><span style='background:#fee0b6;opacity:0.7;'></span>15% to 30%</li>
    <li><span style='background:#d8daeb;opacity:0.7;'></span>30% to 45%</li>
    <li><span style='background:#998ec3;opacity:0.7;'></span>45% to 60%</li>
    <li><span style='background:#8856a7;opacity:0.7;'></span>60% to 75%</li>
    <li><span style='background:#542788;opacity:0.7;'></span>>75%</li>


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edmA_map.get_root().add_child(macro)

edmA_map

### Bar chart

In [ ]:
color_list = ['plum','khaki'] 

ax = family2016.plot.barh(stacked=True,figsize = (20, 20), width = 0.8, color = color_list, align='center', edgecolor = None);



ax.set_title("2019 Family with/without Children Status", fontsize = 16)#use font size 16 for the title
ax.legend(fontsize='14',facecolor='white')#use font size 14 for legend
ax.tick_params(axis='x', labelsize=14)#use font size 14 for axis-lable
ax.set_xlabel('Percentage') 
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(str(width.round(1))+'%', xy=(left+width/2, bottom+height/2), 
                ha='center', va='center')
    
#ax.set_xlim(0, 35)
    
# remove the left, top, and right borders; make sure the color of x axix labels is black
ax.set_facecolor('xkcd:white') # change background to white  #ax.set_facecolor('xkcd:white')
#ax.set_xticklabels([]) # turn off y ticks
ax.tick_params(left=False)
ax.axvline(0, color='black') # draw an x axix line 
ax.tick_params(axis='x', colors='black') # make sure the color of x axix labels is black    

## C7. Import Household Income     
**(Usually, people with higher income spend more on food from restaurant)**    
Reference:    
1. Before income taxes and deductions, during a specified reference period.
2. For the population aged 15 years and over
 - https://data.edmonton.ca/Census/2016-Census-Population-by-Household-Income-Neighbo/jkjx-2hix 2016 Census - Population by Household Income (Neighbourhood/Ward)

In [ ]:
d_income = pd.read_csv("https://data.edmonton.ca/api/views/jkjx-2hix/rows.csv?accessType=DOWNLOAD")
d_income.tail()

### C7.1 Clean data

In [ ]:
d_income.rename(columns={'Neighbourhood Name':'Neighbourhood','Less than $30,000':'X<30K','$30,000 to less than $60,000':'30K≤X<60K','$60,000 to less than $100,000':'60K≤X<100K','$100,000 to less than $125,000':'100K≤X<125K','$125,000 to less than $150,000':'125K≤X<150K','$150,000 to less than $200,000':'150K≤X<200K','$200,000 to less than $250,000':'200K≤X<250K','$250,000 or more':'X≥250K'}, inplace=True)
d_income['Neighbourhood']=d_income['Neighbourhood'].str.title()
d_income.drop(['Ward','Neighbourhood Number'],axis=1, inplace=True)


d_income['Neighbourhood']=d_income['Neighbourhood'].replace({'Mcconachie Area':'McConachie Area','Macewan':'MacEwan','Glenridding Area':'Glenridding Heights','Central Mcdougall': 'Central McDougall'})
d_income.sort_values(['Neighbourhood'], inplace=True ,ascending=True)


income2016 = pd.merge(d_income,
                dtopNei[['Neighbourhood']],
                 on='Neighbourhood')
income2016.tail()

In [ ]:
dm1=income2016.copy()
dm2=dtopNei.copy()
dm1=dm1[['Neighbourhood']]


dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

In [ ]:
#income2016.set_index('Neighbourhood', inplace=True)
income2016.tail()

### Edmonton - Average householde income (based on 2016 census, statCan)
Reference:    
1. Before income taxes and deductions, during a specified reference period.
2. For the population aged 15 years and over
 - https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=CMACA&Code1=835&Geo2=PR&Code2=48&Data=Count&SearchText=edmonton&SearchType=Begins&SearchPR=01&B1=All&TABID=1

In [ ]:
import html5lib

url = 'https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=CMACA&Code1=835&Geo2=PR&Code2=48&SearchText=edmonton&SearchType=Begins&SearchPR=01&B1=Income&TABID=1&type=0'

dfs = pd.read_html(url)
df = dfs[0] #import the n+1th table 
df.reset_index(drop=True, inplace=True)
df.head()

In [ ]:
df.iloc[3][0] #[i+1th row][j+1th column]

In [ ]:
b=df['Characteristic']['Characteristic']['Characteristic']['Characteristic'].str.strip().str[0:6].iloc[3]
print(b)

In [ ]:
 a=df['Characteristic']['Characteristic']['Characteristic']['Characteristic'].str.contains('after-tax', regex=False).iloc[5]
print(a)

In [ ]:
df.iloc[3][1]

In [ ]:
dra=[]
drb=[]
drc=[]
drd=[]
for i in range((len(df['Characteristic']['Characteristic']['Characteristic']['Characteristic'])-1)):
    a=df['Characteristic']['Characteristic']['Characteristic']['Characteristic'].str.contains('after-tax', regex=False).iloc[i]
    if a == False: 
        b=df['Characteristic']['Characteristic']['Characteristic']['Characteristic'].str.strip().str[0:7].iloc[i]
        bb=df.iloc[i][0]
        c=df.iloc[i][1]
        d=df.iloc[i][2]
        e=df.iloc[i][3]
        if b == 'Average':
            dra.append(bb)
            drb.append(c)
            drc.append(d)
            drd.append(e)
        else:
            continue

dataAve = {'Characteristic': dra,
            'Total':drb,
         'Male':drc,
         "Female":drd
        }

dtAve = pd.DataFrame (dataAve, columns = ['Characteristic','Total','Male','Female'])
dtAve.set_index('Characteristic', inplace=True)
dtAve#.tail()

In [ ]:
dra=[]
drb=[]
drc=[]
drd=[]
for i in range((len(df['Characteristic']['Characteristic']['Characteristic']['Characteristic'])-1)):
    a=df['Characteristic']['Characteristic']['Characteristic']['Characteristic'].str.contains('after-tax', regex=False).iloc[i]
    if a == False: 
        b=df['Characteristic']['Characteristic']['Characteristic']['Characteristic'].str.strip().str[0:6].iloc[i]
        bb=df.iloc[i][0]
        c=df.iloc[i][1]
        d=df.iloc[i][2]
        e=df.iloc[i][3]
        if b == 'Median':
            dra.append(bb)
            drb.append(c)
            drc.append(d)
            drd.append(e)
        else:
            continue
    

In [ ]:
dataMed = {'Characteristic': dra,
            'Total':drb,
         'Male':drc,
         "Female":drd
        }

dtMed = pd.DataFrame (dataMed, columns = ['Characteristic','Total','Male','Female'])
dtMed.set_index('Characteristic', inplace=True)
dtMed#.tail()

### Define income class (based on before tax income)

Definition of middle class household (with an income that is 67% to 200% (two-thirds to double) of the overall median household income,Lower-income households have incomes less than 67% of the median, and upper-income households have incomes that are more than double the median.)    
lowest-income households (with income less than half of the overall median) and lower-middle income households (with incomes from half to less than two-thirds of the overall median)    
upper-middle income households (with more than twice the overall median income and up to three times the median) and highest-income households (with more than three times the overall median income)
 - https://www.pewsocialtrends.org/2015/12/09/the-american-middle-class-is-losing-ground/

Median total income of households in 2015 ($)

In [ ]:
# selecting rows based on condition 
rslt_df = dtMed[dtMed.index == 'Median total income of households in 2015 ($)']
rslt_df.reset_index(drop=True, inplace=True)
Med=int(rslt_df.iloc[0][0])
print(Med)

In [ ]:
dataclass = {'Income class':['lowest income households','lower-middle income households','middle income households','upper-middle income households','highest income households'],
            'Income range': ["X<"+"{:.2f}".format(0.5*Med),"{:.2f}".format(0.5*Med)+"≤X<"+"{:.2f}".format(2/3*Med),"{:.2f}".format(2/3*Med)+"≤X<"+"{:.2f}".format(2*Med),"{:.2f}".format(2*Med)+"≤X<"+"{:.2f}".format(3*Med),"{:.2f}".format(3*Med)+"≤X"]
        }

dtclass = pd.DataFrame (dataclass, columns = ['Income class','Income range'])
dtclass.round(2)
dtclass#.tail()

In [ ]:
datacl = {'Boundary of each class':['Upper Boundary','Lower Boundary'],
            'Lower Income Class':[0,2/3*Med],
            'Middle Income Class':[2/3*Med,2*Med],
            'Upper Income Class':[2*Med,float('inf')],
        }

dtcl = pd.DataFrame (datacl, columns = ['Boundary of each class','Lower Income Class','Middle Income Class','Upper Income Class'])
dtcl=dtcl.round(2)
dtcl#.tail()

In [ ]:
income2016c=income2016.copy()
income2016c.drop(['No Response'],axis=1, inplace=True)
cols_to_sum = income2016c.columns[ 0: income2016.shape[1]]
income2016c['Total']=income2016c[cols_to_sum].sum(axis=1)

income2016c['X<30K (%)']=income2016c['X<30K']/income2016c['Total']*100
income2016c['30K≤X<60K (%)']=income2016c['30K≤X<60K']/income2016c['Total']*100

income2016c['60K≤X<100K (%)']=income2016c['60K≤X<100K']/income2016c['Total']*100
income2016c['100K≤X<125K (%)']=income2016c['100K≤X<125K']/income2016c['Total']*100
income2016c['125K≤X<150K (%)']=income2016c['125K≤X<150K']/income2016c['Total']*100
income2016c['150K≤X<200K (%)']=income2016c['150K≤X<200K']/income2016c['Total']*100

income2016c['200K≤X<250K (%)']=income2016c['200K≤X<250K']/income2016c['Total']*100
income2016c['X≥250K (%)']=income2016c['X≥250K']/income2016c['Total']*100

income2016c['Lower-class (%)']=income2016c['X<30K (%)']+income2016c['30K≤X<60K (%)']
income2016c['Middle-class (%)']=income2016c['60K≤X<100K (%)']+income2016c['100K≤X<125K (%)']+income2016c['125K≤X<150K (%)']+income2016c['150K≤X<200K (%)']
income2016c['Upper-class (%)']=income2016c['200K≤X<250K (%)']+income2016c['X≥250K (%)']
#income2016c['Percentage No Response (%)']=income2016c['No Response']/income2016c['Total']*100


#income2016.sort_values(['Percentage Owned (%)'], inplace=True ,ascending=True)
#income2016.reset_index(drop=True, inplace=True)

income2016c.tail()

In [ ]:
#check whether respondse% over 25% of the total population
check = pd.merge(income2016c,
                dN_2016n2[['Neighbourhood','Total']],
                 on='Neighbourhood')
check['% respond']=check['Total_x']/check['Total_y']

In [ ]:
D2016inc=income2016c.copy()
D2016inc.iloc[1][(income2016.shape[1]):(income2016c.shape[1]-3)].index

In [ ]:
D2016inc=income2016c.copy()
#D2016inc.drop(['X<30K','30K≤X<60K','60K≤X<100K','100K≤X<125K','125K≤X<150K','150K≤X<200K','200K≤X<250K','X≥250K','Total','Bottom-class (%)','Middle-class (%)','Highest-class (%)'],axis=1, inplace=True)D2016inc.sort_values(['X<30K (%)'], inplace=True ,ascending=True)
D2016inc=D2016inc[D2016inc.iloc[1][(income2016.shape[1]):(income2016c.shape[1]-3)].index]
D2016inc['Neighbourhood']=income2016c['Neighbourhood']
D2016inc.sort_values(['X<30K (%)'], inplace=True ,ascending=True)
D2016inc=D2016inc.dropna()
#D2016inc.reset_index(drop=True, inplace=True)
D2016inc.set_index('Neighbourhood', inplace=True)
D2016inc.tail()

In [ ]:
D2016inclass=income2016c.copy()
D2016inclass=D2016inclass[['Neighbourhood','Middle-class (%)','Lower-class (%)','Upper-class (%)']]
D2016inclass.sort_values(['Middle-class (%)'], inplace=True ,ascending=True)
D2016inclass=D2016inclass.dropna()
D2016inclass.reset_index(drop=True, inplace=True)
D2016inclass.set_index('Neighbourhood', inplace=True)
D2016inclass.tail()

### Heat map

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisA.tail()

In [ ]:
NeisA.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisA,
                dtopNei,
                 on='Neighbourhood')
NeiAmerge = pd.merge(NeiAmerge,
                D2016inclass,
                 on='Neighbourhood')
NeiAmerge.rename(columns={'Neighbourhood':'name'}, inplace=True)
NeiAmerge['centroid_long']=NeiAmerge.centroid.x
NeiAmerge['centroid_lag']=NeiAmerge.centroid.y
NeiAmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09#PuOr
colormap = cm.linear.BrBG_09.scale(-20,100).to_step(6) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron') #, tiles = 'Stamen Toner')
colormap.caption = "Middle-class (%)"

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']["Middle-class (%)"]), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name',"Middle-class (%)"],
        aliases=['name',"Middle-class (%)"],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

colormap.add_to(edmA_map)

from branca.element import Template, MacroElement
class DivIcon(MacroElement):
    def __init__(self, html='', size=(5,5), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)
for i in range(len(NeiAmerge)):
    lag=NeiAmerge.iloc[i][8]
    log=NeiAmerge.iloc[i][7]
    text=NeiAmerge.iloc[i][2]
    folium.map.Marker(
        [lag+0.003, log-0.005],
        icon=DivIcon(
            size=(5,5),
            anchor=(0,0),
            html=text,
            style="""
            font-size:8px;
            background-color: transparent;
            border-color: transparent;
            text-align: center;
            """
        )
    ).add_to(edmA_map)
edmA_map

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Middle-class (%)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#d8b365;opacity:0.7;'></span><20%</li>
    <li><span style='background:#f6e8c3;opacity:0.7;'></span>20% to 40%</li>
    <li><span style='background:#c7eae5;opacity:0.7;'></span>40% to 60%</li>
    <li><span style='background:#5ab4ac;opacity:0.7;'></span>60% to 80%</li>
    <li><span style='background:#01665e;opacity:0.7;'></span>>80%</li>


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edmA_map.get_root().add_child(macro)

edmA_map

### Bar chart

import seaborn as sns
fig, ax = plt.subplots(figsize=(20,20))
ax = sns.heatmap(D2016inclass, annot =D2016inclass.round(2), fmt = '',linewidths=1, linecolor='white',cbar=False,cmap="YlGnBu",)

plt.xlabel("Househole Income")

In [ ]:
color_list = ['plum','khaki','lemonchiffon'] 

ax = D2016inclass.plot.barh(stacked=True,figsize = (20, 20), width = 0.8, color = color_list, align='center', edgecolor = None);



ax.set_title("2019 Househole Income Status", fontsize = 16)#use font size 16 for the title
ax.legend(fontsize='14',facecolor='white')#use font size 14 for legend
ax.tick_params(axis='x', labelsize=14)#use font size 14 for axis-lable
ax.set_xlabel('Percentage') 
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(str(width.round(1))+'%', xy=(left+width/2, bottom+height/2), 
                ha='center', va='center')
    
#ax.set_xlim(0, 35)
    
# remove the left, top, and right borders; make sure the color of x axix labels is black
ax.set_facecolor('xkcd:white') # change background to white  #ax.set_facecolor('xkcd:white')
#ax.set_xticklabels([]) # turn off y ticks
ax.tick_params(left=False)
ax.axvline(0, color='black') # draw an x axix line 
ax.tick_params(axis='x', colors='black') # make sure the color of x axix labels is black    

## C8. Import Highest Education Status       
Usually, people with higher education level tends to be more frequently consumpt alcohol in Canada
Reference:    
 - https://data.edmonton.ca/Census/2016-Census-Population-by-Highest-Educational-Atta/f7ms-98xd 2016 Census - Population by Highest Educational Attainment (Neighbourhood/Ward)
 - https://www.statista.com/statistics/895733/frequency-of-alcohol-consumption-by-education-canada/ 
 - http://angusreid.org/wp-content/uploads/2018/08/2018.07.16-alcohol-ReleaseTables.pdf    
 How frequently Canadians drink alcohol by education 2018    
 - https://www.nature.com/articles/s41380-019-0535-9 
 Increased EA are not associated with total amount of weekly drinks, they are associated with reduced frequency of binge drinking ≥6 drinks, reduced total drinks consumed per drinking day, as well as lower weekly distilled spirits intake.
 Increased EA were associated with increased alcohol intake frequency, and increased weekly white wine and red wine intake

In [ ]:
d_edu = pd.read_csv("https://data.edmonton.ca/api/views/f7ms-98xd/rows.csv?accessType=DOWNLOAD")
d_edu.head()

### C8.1 Clean data

In [ ]:
d_edu.rename(columns={'Neighbourhood Name':'Neighbourhood'}, inplace=True)
d_edu['Neighbourhood']=d_marital['Neighbourhood'].str.title()
d_edu.drop(['Ward','Neighbourhood Number'],axis=1, inplace=True)

d_edu['Neighbourhood']=d_edu['Neighbourhood'].replace({'Macewan':'MacEwan','Glenridding Area':'Glenridding Heights','Central Mcdougall': 'Central McDougall'})
d_edu.sort_values(['Neighbourhood'], inplace=True ,ascending=True)

edu2016 = pd.merge(d_edu,
                dtopNei[['Neighbourhood']],
                 on='Neighbourhood')
edu2016.tail()

In [ ]:
dm1=edu2016.copy()
dm2=dtopNei.copy()
dm1=dm1[['Neighbourhood']]


dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

edu2016.set_index('Neighbourhood', inplace=True)
edu2016.tail()

### Defination of each education level
Reference:
 - https://www12.statcan.gc.ca/nhs-enm/2011/ref/dict/pop038-eng.cfm
 - https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVDStruct&TVD=306216&CVD=306218&CPV=2.2&CST=07042016&CLV=1&MLV=6

In [ ]:
edu2016c=edu2016.copy()
edu2016c.iloc[1][0:edu2016.shape[1]].index

In [ ]:
edu2016c=edu2016.copy()
edu2016c.drop(['No Response'],axis=1, inplace=True)
cols_to_sum = edu2016c.columns[ 0: edu2016c.shape[1]]
edu2016c['Total']=edu2016c[cols_to_sum].sum(axis=1)

edu2016c['High school or less (%)']=(edu2016c['No Certificate, Diploma or Degree']+edu2016c['High School Diploma or Equivalent'])/edu2016c['Total']*100
edu2016c['College/Tech (%)']=(edu2016c['Registered Apprenticeship Certificate']+edu2016c['Trades Certificate']+edu2016c['College Certificate or Diploma'])/edu2016c['Total']*100
edu2016c['University+ (%)']=(edu2016c["University Certificate Below Bachelor's Level"]+edu2016c["Bachelor's Degree"]+edu2016c["University Certificate or Diploma above Bachelor Level"]+edu2016c["Medical Degree"]+edu2016c["Master's Degree"]+edu2016c["Earned Doctorate"])/edu2016c['Total']*100

edu2016c=edu2016c[['Neighbourhood','University+ (%)','College/Tech (%)','High school or less (%)']]
edu2016c.sort_values(['University+ (%)'], inplace=True ,ascending=True)
edu2016c.reset_index(drop=True, inplace=True)
edu2016c.tail()

In [ ]:
edu2016cc=edu2016c.copy()
edu2016cc=edu2016cc.dropna()
edu2016cc.reset_index(drop=True, inplace=True)
edu2016cc.set_index('Neighbourhood', inplace=True)
edu2016cc.tail()

### Since the data from city is not complete, import data from Federal census 2016

Reference：
 - https://public.tableau.com/profile/city.of.edmonton#!/vizhome/NeighbourhoodProfiles_FederalCensus2016/FamilyandLanguage
1. Download Tableau from the website
2. Access data by clicking 'data' from tableau and explore all into csv
3. upload to IBM_Cloud→Db2-vs→Manage→Open console→Load data (under GLN55437)

### Import new set of data

###### M1c.1 Connect to the database from SQL

Import data from SQL SERVER

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from Education"

#retrieve the query results into a pandas dataframe
f_edu = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
f_edu.head(6)

### Clean data (Federal census)

In [ ]:
f_edu.rename(columns={'Name':'Neighbourhood','Education_Level_PT':'Education Level (%)'}, inplace=True)
f_edu['Neighbourhood']=f_edu['Neighbourhood'].str.title()
f_edu.drop(['Nbhd_Num','COE_Education_Level_Count','COE_Education_Level_PT','Education_Level_Count'],axis=1, inplace=True)
f_edu['Neighbourhood']=f_edu['Neighbourhood'].replace({'Macewan':'MacEwan','Central Mcdougall': 'Central McDougall','Mcconachie Area':'McConachie Area'})
f_edu.sort_values(['Neighbourhood'], inplace=True ,ascending=True)


edu2016 = pd.merge(f_edu,
                      dtopNei[['Neighbourhood']],
                      on='Neighbourhood')
edu2016.head(10)

In [ ]:
dm1=edu2016.copy()
dm2=dtopNei.copy()
dm1=dm1.groupby('Neighbourhood',axis=0)['Education Level (%)'].sum().reset_index()
dm1=dm1[['Neighbourhood']]
dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

In [ ]:
S_edu2016=edu2016.copy()
S_edu2016=S_edu2016.pivot(index='Neighbourhood',columns='Education_Level',values='Education Level (%)')

S_edu2016=S_edu2016.astype(float)

S_edu2016['University+ (%)']=S_edu2016['University certificate or diploma below bachelor level']+S_edu2016['University certificate, diploma or degree at bachelor level or above']
S_edu2016['College/Tech (%)']=S_edu2016['Apprenticeship or trades certificate or diploma']+S_edu2016['College, CEGEP or other non-university certificate or diploma']
S_edu2016['High school or less (%)']=S_edu2016['High school diploma or equivalency certificate']+S_edu2016['No certificate, diploma or degree']
S_edu2016.sort_values(['University+ (%)'], inplace=True ,ascending=True)

S_edu2016.head()

In [ ]:
a=S_edu2016.copy()
a=a['University+ (%)'].reset_index()
b=S_edu2016.copy()
b=b['College/Tech (%)'].reset_index()
c=S_edu2016.copy()
c=c['High school or less (%)'].reset_index()
education2016 = pd.merge(a,
                b,
                 on='Neighbourhood')
education2016 = pd.merge(education2016,
                c,
                 on='Neighbourhood')
education2016.set_index('Neighbourhood', inplace=True)
education2016.tail()

### Heat map

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisA.tail()

In [ ]:
NeisA.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisA,
                dtopNei,
                 on='Neighbourhood')
NeiAmerge = pd.merge(NeiAmerge,
                education2016,
                 on='Neighbourhood')
NeiAmerge.rename(columns={'Neighbourhood':'name'}, inplace=True)
NeiAmerge['centroid_long']=NeiAmerge.centroid.x
NeiAmerge['centroid_lag']=NeiAmerge.centroid.y
NeiAmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09#PuOr
colormap = cm.linear.RdYlBu_06.scale(0,90).to_step(6) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron') #, tiles = 'Stamen Toner')
colormap.caption = "University+ Education(%)"

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']["University+ (%)"]), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name',"University+ (%)"],
        aliases=['name',"University+ (%)"],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

colormap.add_to(edmA_map)

from branca.element import Template, MacroElement
class DivIcon(MacroElement):
    def __init__(self, html='', size=(5,5), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)
for i in range(len(NeiAmerge)):
    lag=NeiAmerge.iloc[i][8]
    log=NeiAmerge.iloc[i][7]
    text=NeiAmerge.iloc[i][2]
    folium.map.Marker(
        [lag+0.003, log-0.005],
        icon=DivIcon(
            size=(5,5),
            anchor=(0,0),
            html=text,
            style="""
            font-size:8px;
            background-color: transparent;
            border-color: transparent;
            text-align: center;
            """
        )
    ).add_to(edmA_map)
edmA_map

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>University+ (%)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#d73027;opacity:0.7;'></span><15%</li>
    <li><span style='background:#fc8d59;opacity:0.7;'></span>15% to 30%</li>
    <li><span style='background:#fee090;opacity:0.7;'></span>30% to 45%</li>
    <li><span style='background:#e0f3f8;opacity:0.7;'></span>45% to 60%</li>
    <li><span style='background:#91bfdb;opacity:0.7;'></span>60% to 75%</li>
    <li><span style='background:#4575b4;opacity:0.7;'></span>>75%</li>


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edmA_map.get_root().add_child(macro)

edmA_map

### Bar chart

In [ ]:
color_list = ['plum','khaki','lemonchiffon'] 

ax = education2016.plot.barh(stacked=True,figsize = (20, 20), width = 0.8, color = color_list, align='center', edgecolor = None);



ax.set_title("2019 Education Status", fontsize = 16)#use font size 16 for the title
ax.legend(fontsize='14',facecolor='white',loc="lower right")#use font size 14 for legend
ax.tick_params(axis='x', labelsize=14)#use font size 14 for axis-lable
ax.set_xlabel('Percentage') 
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(str(width.round(1))+'%', xy=(left+width/2, bottom+height/2), 
                ha='center', va='center')
    
#ax.set_xlim(0, 35)
    
# remove the left, top, and right borders; make sure the color of x axix labels is black
ax.set_facecolor('xkcd:white') # change background to white  #ax.set_facecolor('xkcd:white')
#ax.set_xticklabels([]) # turn off y ticks
ax.tick_params(left=False)
ax.axvline(0, color='black') # draw an x axix line 
ax.tick_params(axis='x', colors='black') # make sure the color of x axix labels is black    


## C9. Commercial accessment lot size
1. larger lot size, more people in that are
2. The unit of Alberta registered lot size is in meters squared

### Obtain commercial zoning ID
Reference:
 - https://webdocs.edmonton.ca/InfraPlan/zoningbylaw/bylaw_12800.htm
 - https://data.edmonton.ca/Thematic-Features/City-of-Edmonton-Zoning-Bylaw-Map/b4f2-gf2b City of Edmonton Zoning Bylaw

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/b4f2-gf2b?method=export&format=GeoJSON") #provde by city of Edmonton

data = response.json()
BIA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

BIA.head()

In [ ]:
Busi=BIA[BIA.descript.str.contains("Commercial")]
Busi=Busi.append(BIA[BIA.descript.str.contains("Business")], ignore_index=True)
Busi=Busi.append(BIA[BIA.descript.str.contains("Corridor")], ignore_index=True)
Busi=Busi.append(BIA[BIA.descript.str.contains("Shopping")], ignore_index=True)
Busi=Busi.append(BIA[BIA.zoning.str.contains("UW")], ignore_index=True)
Busi=Busi.append(BIA[BIA.descript.str.contains("Griesbach Village Centre Zone")], ignore_index=True)
Busi=Busi.append(BIA[BIA.descript.str.contains("Heritage Area Zone")], ignore_index=True)
Busi=Busi.append(BIA[BIA.descript.str.contains("Marquis")], ignore_index=True)
Busi=Busi.append(BIA[BIA.descript.str.contains("Mixed Use Zone")], ignore_index=True)
Busi=Busi.append(BIA[BIA.descript.str.contains("Arena and Entertainment District Zone")], ignore_index=True)

Busi=Busi[~Busi.descript.str.contains("Industrial")]
Busi=Busi[~Busi.descript.str.contains("Residential Mixed Use Zone")]
Busi=Busi[~Busi.zoning.str.contains("MA3")]


Busi=Busi.groupby('zoning',axis=0)['id'].count().reset_index()
Busi= pd.merge(Busi, BIA[['zoning','descript']],on='zoning')
Busi.drop_duplicates(subset="zoning", keep = 'first', inplace = True) 
Busi.reset_index(drop=True, inplace=True)
Busi.drop('id',axis=1, inplace=True)
Busi

### Assessment.2019

Reference:
 - https://data.edmonton.ca/City-Administration/Property-Assessment-Data-2012-2019-/qi6a-xuwt Property Assessment Data (2012 - 2019)
 - https://data.edmonton.ca/City-Administration/Property-Assessment-Data-Current-Calendar-Year-/q7d6-ambg Property Assessment Data (Current Calendar Year)
 - https://data.edmonton.ca/City-Administration/Property-Information-Data-Current-Calendar-Year-/dkk9-cj3x Property Information Data (It is effective from January 1st, 2017 until December 31st, 2017)

In [ ]:
access=pd.read_csv("Accessment_EDMC.csv") 

In [ ]:
access.dtypes

In [ ]:
dmiss1 = pd.DataFrame({"zoning":['CMX*', 'DC1', 'DC2','DC3','DC4','DC5'], 
                    "descript":['Not assigned','Direct Development Control Provisions', 'Site Specific Development Control Provision','Not assigned','Not assigned','Not assigned'],  
                    }) 
dmiss1

In [ ]:
import math
diser_zoning=Busi.append([dmiss1], ignore_index=True)
access2019=pd.merge(diser_zoning, access,on='zoning')
access2019=access2019[access2019['Assessment Year']==2019]

access2019["House Number"]=access2019["House Number"].astype('Int64')
access2019["Adress"] = access2019["House Number"].astype(str) +str(' ')+ access2019["Street Name"]
access2019.drop(['Suite','Legal Description','Garage','New Georeferenced Column','Assessment Class % 1','Assessment Class % 2','Assessment Class % 3'],axis=1, inplace=True)

access2019=access2019[access2019['Assessment Class 1'].str.contains("COMMERCIAL")]
access2019=access2019[~access2019['Assessment Class 2'].str.contains("FARMLAND", na=False)]
access2019=access2019[~access2019['Assessment Class 3'].str.contains("NONRES MUNICIPAL/RES EDUCATION", na=False)]
#access2019 = access2019[access2019['Street Name'].notnull()]
access2019.reset_index(drop=True, inplace=True)
access2019.head()

In [ ]:
class2=access2019.groupby('Assessment Class 2',axis=0)['Account Number'].count().reset_index()
class2

In [ ]:
class3=access2019.groupby('Assessment Class 3',axis=0)['Account Number'].count().reset_index()
class3

In [ ]:
access2019.to_csv("Accessment2019.csv",index=False)

In [ ]:
Total_lotsize=access2019.groupby('Neighbourhood',axis=0)['Lot Size'].sum().reset_index()
Total_lotsize.sort_values(['Lot Size'], inplace=True ,ascending=False)
Total_lotsize.reset_index(drop=True, inplace=True)
Total_lotsize.head()

In [ ]:
Num_Business=access2019.groupby('Neighbourhood',axis=0)['Account Number'].count().reset_index()
Num_Business.rename(columns={'Account Number':'Commercial Assessment Number'}, inplace=True)
Num_Business.sort_values(['Commercial Assessment Number'], inplace=True ,ascending=False)
Num_Business.reset_index(drop=True, inplace=True)
Num_Business.head()

In [ ]:
commercial_acess2019 = pd.merge(Total_lotsize,
                dtopNei[['Neighbourhood']],
                 on='Neighbourhood')
commercial_acess2019= pd.merge(commercial_acess2019,
                Num_Business,
                 on='Neighbourhood')
commercial_acess2019.sort_values(['Lot Size'], inplace=True ,ascending=True)
commercial_acess2019.reset_index(drop=True, inplace=True)
commercial_acess2019.tail()

### Neighbourhood with ≥10 Millennials per net residential hectare but without commercial assessment by 2019

In [ ]:
dm1=commercial_acess2019.copy()
dm2=dtopNei.copy()
dm1=dm1[['Neighbourhood']]


dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

In [ ]:
dmiss['Lot Size']=[0]*len(dmiss)
dmiss['Commercial Assessment Number']=[0]*len(dmiss)

In [ ]:
commercial2019=dmiss.append([commercial_acess2019], ignore_index=True)

### LRT

Reference:
 - https://public.tableau.com/profile/city.of.edmonton#!/vizhome/2019LRTPassengerCountReport_15886048332290/LRTSurvey

###### M1c.1 Connect to the database from SQL

Import data from SQL SERVER

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from LRT_ACTIVITY1819"

#retrieve the query results into a pandas dataframe
f_lrt = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first 5 rows in the pandas data frame
f_lrt.tail()

### Clean data (Federal census)

In [ ]:
f_lrt=f_lrt[['LRT_Station','2019 Station Total']]
f_lrt=f_lrt[f_lrt['LRT_Station']!='TOTAL LRT Station']
f_lrt.tail()

geolocator = Nominatim(user_agent='cgao2@ualberta.ca')
rec1=[]
rec2=[]
rec3=[]
for i in range((len(f_lrt)-2)):
    a=f_lrt.iloc[i][0]+', Edmonton, Alberta, Canada'
    if a=='South Campus/Ft Edmonton Station, Edmonton, Alberta, Canada':
        a='University of alberta South Campus, Edmonton, Alberta, Canada'
    elif a=='Grandin/Govt Centre Station, Edmonton, Alberta, Canada':
        a='Grandin Station, Edmonton, Alberta, Canada'
    location = geolocator.geocode(a)
    rec1.append(location.address)
    rec2.append(location.latitude)
    rec3.append(location.longitude)

datalrt = {'Address': rec1,
            'Latitude':rec2,
         'Longitude':rec3
        }
dlrt = pd.DataFrame (datalrt, columns = ['Address','Latitude','Longitude'])
dlrt.tail()

### Heat map Neighbourhood ≥ 10 Millennials per net residential hectare & lrt stations

Get the geographical coordinates for each borough in Edmonton

In [ ]:
address = 'Edmonton, AB'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Edmonton are {}, {}.'.format(latitude, longitude))

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisA.tail()

In [ ]:
NeisA.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisA,
                dtopNei,
                 on='Neighbourhood')
NeiAmerge.rename(columns={'Neighbourhood':'name'}, inplace=True)
NeiAmerge['Num']=[1]*len(NeiAmerge)
NeiAmerge['centroid_long']=NeiAmerge.centroid.x
NeiAmerge['centroid_lag']=NeiAmerge.centroid.y
NeiAmerge.tail()

In [ ]:
import geopandas as gpd
response = requests.get(r"https://raw.githubusercontent.com/cgao2/BIA-map-/master/LRT.geojson") #create by me

data = response.json()
lrt = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')
lrt['Num']=[2]*len(lrt)
lrt.head()

In [ ]:
NeiLRTmerge = lrt.append(NeiAmerge, ignore_index=True)

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09#PuOr
colormap = cm.linear.PiYG_04.scale(0,1)#.to_step(1) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron') #, tiles = 'Stamen Toner')
#colormap.caption = "Numbers of Millennials per net residential hectareper"

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['Num']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



g = folium.GeoJson(
    NeiLRTmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name','Num'],
        aliases=['name','Num'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

#colormap.add_to(edmA_map)

from branca.element import Template, MacroElement
class DivIcon(MacroElement):
    def __init__(self, html='', size=(5,5), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)
for i in range(len(NeiAmerge)):
    lag=NeiAmerge.iloc[i][6]
    log=NeiAmerge.iloc[i][5]
    text=NeiAmerge.iloc[i][2]
    folium.map.Marker(
        [lag, log],
        icon=DivIcon(
            size=(5,5),
            anchor=(0,0),
            html=text,
            style="""
            font-size:8px;
            background-color: transparent;
            border-color: transparent;
            text-align: center;
            """
        )
    ).add_to(edmA_map)
edmA_map

## C-Final. Score selected neighbourhood for fine dinning restuarant or fast casual restuarant

In [ ]:
Mdensity2019=topNei_Mdensity.copy()
Mdensity2019['Millennials density score']=Mdensity2019['Millennials Per Net Residential Hectare']/8
Mdensity2019['Millennials density score']=Mdensity2019['Millennials density score'].apply(np.floor)
#Mdensity2019=Mdensity2019[['Neighbourhood','Millennials density score']]
Mdensity2019.reset_index(drop=True, inplace=True)
Mdensity2019.tail()

In [ ]:
occupy2019=D2019Ds.copy()
occupy2019=occupy2019[['Percentage Occupied (%)']]
occupy2019=occupy2019.reset_index()
occupy2019['Dwelling occupied score']=(occupy2019['Percentage Occupied (%)']/100).round(2)
#occupy2019=occupy2019[['Neighbourhood','Dwelling occupied score']]
Mdensity2019.reset_index(drop=True, inplace=True)
occupy2019.tail()

In [ ]:
ownership2016=D2016DO.copy()
ownership2016=ownership2016[['Owned (%)']]
ownership2016=ownership2016.reset_index()
ownership2016['Dwelling Ownership score']=(ownership2016['Owned (%)']/100).round(2)
#ownership2016=ownership2016[['Neighbourhood','Dwelling Ownership score']]
ownership2016.reset_index(drop=True, inplace=True)
ownership2016.tail()

In [ ]:
mortgage2016=morg2016.copy()
mortgage2016=mortgage2016[['% of HH with a mortgage']]
mortgage2016=mortgage2016.reset_index()
mortgage2016['Dwelling mortgage score']=(mortgage2016['% of HH with a mortgage']/400).round(2)
#mortgage2016=mortgage2016[['Neighbourhood','Dwelling mortgage score']]
mortgage2016.reset_index(drop=True, inplace=True)
mortgage2016.tail()

In [ ]:
marital2016=marital2016c.copy()
marital2016=marital2016[['1 person household (%)']]
marital2016=marital2016.reset_index()
marital2016['Single score']=(marital2016['1 person household (%)']/100*3).round(2)
#marital2016=marital2016[['Neighbourhood','Single score']]
marital2016.reset_index(drop=True, inplace=True)
marital2016.tail()

In [ ]:
children2016=family2016.copy()
children2016=children2016[['Without children (%)']]
children2016=children2016.reset_index()
children2016['Without children score']=(children2016['Without children (%)']/100*3).round(2)
#children2016=children2016[['Neighbourhood','Without children score']]
children2016.reset_index(drop=True, inplace=True)
children2016.tail()

In [ ]:
income2016=D2016inclass.copy()
income2016=income2016[['Middle-class (%)']]#,'Upper-class (%)']]
income2016=income2016.reset_index()
income2016['Middle-class score']=(income2016['Middle-class (%)']/100*3).round(2)
#income2016=income2016[['Neighbourhood','Middle-class score']]
income2016.reset_index(drop=True, inplace=True)
income2016.tail()

In [ ]:
educ2016=education2016.copy()
educ2016=educ2016[['University+ (%)']]
educ2016=educ2016.reset_index()
educ2016['Education level score']=(educ2016['University+ (%)']/100*4).round(2)
#educ2016=educ2016[['Neighbourhood','Education level score']]
educ2016.reset_index(drop=True, inplace=True)
educ2016.tail()

In [ ]:
urban2017=dclassMED.copy()
urban2017['Urban score']=urban2017['Classification']/4
urban2017.loc[urban2017['Urban score'] >= 1, 'Urban score'] = 0.75
urban2017.loc[urban2017['Classification'] == 1, 'Classification'] = 'Rural'
urban2017.loc[urban2017['Classification'] == 2, 'Classification'] = 'Rural'
urban2017.loc[urban2017['Classification'] == 4, 'Classification'] = 'Suburban'
urban2017.loc[urban2017['Classification'] == 5, 'Classification'] = 'Urban'
urban2017.loc[urban2017['Classification'] == 6, 'Classification'] = 'Urban'
#urban2017=urban2017[['Neighbourhood','Urban score']]
urban2017.reset_index(drop=True, inplace=True)
urban2017.tail()

In [ ]:
Business_lotsize2019=commercial2019.copy()
Business_lotsize2019=Business_lotsize2019.reset_index()

Business_lotsize2019['Business score1']=(Business_lotsize2019['Lot Size']/100000).round(2)
Business_lotsize2019.loc[Business_lotsize2019['Business score1'] >= 2, 'Business score1'] = 2
Business_lotsize2019['Business score2']=(Business_lotsize2019['Commercial Assessment Number']/150).round(2)
Business_lotsize2019.loc[Business_lotsize2019['Business score2'] >= 2, 'Business score2'] = 2
Business_lotsize2019['Business score']=(Business_lotsize2019['Business score1']+Business_lotsize2019['Business score2'])/2
Business_lotsize2019.sort_values(['Business score'], inplace=True ,ascending=False)
Business_lotsize2019.reset_index(drop=True, inplace=True)
Business_lotsize2019.tail(10)

In [ ]:
nearlrt = {'Neighbourhood': ['Blue Quill','Ermineskin','Skyrattler',
                             'Malmo Plains','Empire Park',
                             'Queen Alexandra','Garneau',
                             'Garneau','Strathcona',
                             'Garneau','Strathcona',
                             'Oliver',
                             'Downtown',
                             'Downtown',
                             'Downtown',
                             'Downtown','Boyle Street',
                             'Central McDougall',
                             'Central McDougall',
                             'Boyle Street','Cromdale','Parkdale',
                             'Parkdale',
                             'Clareview Town Centre'],
            'LRT_Station':['Century Park Station','Century Park Station','Century Park Station',
                         'Southgate Station','Southgate Station',
                         'McKernan/Belgravia Station','McKernan/Belgravia Station',
                         'Health Sciences/Jubilee Station','Health Sciences/Jubilee Station',
                        'University Station','University Station',
                        'Grandin/Govt Centre Station',
                        'Corona Station',
                        'Bay/Enterprise Square Station',
                        'Central Station',
                        'Churchill Station','Churchill Station',
                        'MacEwan Station',
                        'Kingsway/RAH Station',
                        'Stadium Station','Stadium Station','Stadium Station',
                        'Coliseum Station',
                        'Clareview Station']}
nlrt = pd.DataFrame (nearlrt, columns = ['Neighbourhood','LRT_Station'])
lrtscore = pd.merge(nlrt,f_lrt,on='LRT_Station')
lrtscore['2019 Station Total']=lrtscore['2019 Station Total'].astype(float)
lrtscore=lrtscore.groupby('Neighbourhood',axis=0)['2019 Station Total'].sum().reset_index()

dm1=lrtscore.copy()
dm2=dtopNei.copy()
dm1=dm1[['Neighbourhood']]
dm2=dm2[['Neighbourhood']]
dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]
dmiss.reset_index(drop=True, inplace=True)
dmiss['2019 Station Total']=[0]*len(dmiss)
dmiss.tail()

lrtscore2019=lrtscore.append(dmiss, ignore_index=True)

lrtscore2019.sort_values(['2019 Station Total'], inplace=True ,ascending=False)
lrtscore2019['LRT score']=(lrtscore2019['2019 Station Total']/10000).round(2)
lrtscore2019.loc[lrtscore2019['LRT score'] >= 2, 'LRT score'] = 2
lrtscore2019.reset_index(drop=True, inplace=True)
lrtscore2019.head(15)

In [ ]:
score2016 = pd.merge(ownership2016[['Neighbourhood','Owned (%)']],
                     marital2016[['Neighbourhood','1 person household (%)']],on='Neighbourhood')
score2016 = pd.merge(score2016,income2016[['Neighbourhood','Middle-class (%)']],on='Neighbourhood')
score2016 = pd.merge(score2016,educ2016[['Neighbourhood','University+ (%)']],on='Neighbourhood')
score2016 = pd.merge(score2016,children2016[['Neighbourhood','Without children (%)']],on='Neighbourhood')
score2016 = pd.merge(score2016,occupy2019[['Neighbourhood','Percentage Occupied (%)']],on='Neighbourhood')
score2016 = pd.merge(score2016,mortgage2016[['Neighbourhood','% of HH with a mortgage']],on='Neighbourhood')
score2016 = pd.merge(score2016,urban2017[['Neighbourhood','Classification']],on='Neighbourhood')
score2016 = pd.merge(score2016,Mdensity2019[['Neighbourhood','Millennials Per Net Residential Hectare']],on='Neighbourhood')
score2016 = pd.merge(score2016,Business_lotsize2019[['Neighbourhood','Lot Size','Commercial Assessment Number']],on='Neighbourhood')
score2016 = pd.merge(score2016,lrtscore2019[['Neighbourhood','2019 Station Total']],on='Neighbourhood')
score2016.rename(columns={'2019 Station Total':'2019 LRT Passanger Boarding/Alighting',
                          'Owned (%)':'Dwelling owned (%)',
                          'Lot Size':"Commercial Lot Size (㎡)",
                          '% of HH with a mortgage':'Household with Mortgage (%)',
                          'Percentage Occupied (%)':'Dwelling Occupied (%)'
                          }, inplace=True)
score2016=score2016.round(2)

score2016.drop_duplicates(subset ="Neighbourhood", 
                     keep = False, inplace = True) 

score2016.sort_values(['Neighbourhood'], inplace=True ,ascending=True)
score2016.reset_index(drop=True, inplace=True)
columns_titles = ['Neighbourhood','Millennials Per Net Residential Hectare',
                  'Commercial Lot Size (㎡)','Commercial Assessment Number',
                  '2019 LRT Passanger Boarding/Alighting','University+ (%)',
                  'Middle-class (%)','1 person household (%)','Without children (%)',
                  'Dwelling owned (%)','Household with Mortgage (%)',
                  'Dwelling Occupied (%)','Classification']
score2016=score2016.reindex(columns=columns_titles)
score2016#.head(10)

In [ ]:
score2016 = pd.merge(ownership2016[['Neighbourhood','Dwelling Ownership score']],marital2016[['Neighbourhood','Single score']],on='Neighbourhood')
score2016 = pd.merge(score2016,income2016[['Neighbourhood','Middle-class score']],on='Neighbourhood')
score2016 = pd.merge(score2016,educ2016[['Neighbourhood','Education level score']],on='Neighbourhood')
score2016 = pd.merge(score2016,children2016[['Neighbourhood','Without children score']],on='Neighbourhood')
score2016 = pd.merge(score2016,occupy2019[['Neighbourhood','Dwelling occupied score']],on='Neighbourhood')
score2016 = pd.merge(score2016,mortgage2016[['Neighbourhood','Dwelling mortgage score']],on='Neighbourhood')
score2016 = pd.merge(score2016,urban2017[['Neighbourhood','Urban score']],on='Neighbourhood')
score2016 = pd.merge(score2016,Mdensity2019[['Neighbourhood','Millennials density score']],on='Neighbourhood')
score2016 = pd.merge(score2016,Business_lotsize2019[['Neighbourhood','Business score']],on='Neighbourhood')
score2016 = pd.merge(score2016,lrtscore2019[['Neighbourhood','LRT score']],on='Neighbourhood')

#score2016['University+ (%)'] = score2016['University+ (%)'].fillna(0)
cols_to_sum = score2016.columns[ 1: score2016.shape[1]]
score2016['Total Score']=score2016[cols_to_sum].sum(axis=1)
score2016=score2016.round(2)

score2016.drop_duplicates(subset ="Neighbourhood", 
                     keep = False, inplace = True) 

score2016.sort_values(['Total Score'], inplace=True ,ascending=False)
score2016.reset_index(drop=True, inplace=True)
columns_titles = ['Neighbourhood','Millennials density score','Business score','LRT score','Education level score','Middle-class score','Single score','Without children score','Dwelling Ownership score','Dwelling mortgage score','Dwelling occupied score','Urban score','Total Score']
score2016=score2016.reindex(columns=columns_titles)
score2016#.head(10)

In [ ]:
sum_score=score2016.describe().round(2)
sum_score

In [ ]:
desireN=score2016.copy()
desireN = desireN[desireN['Education level score'] >= 1.8] #over 45% of the population in the selected neighbourhood have University + education
desireN = desireN[desireN['Middle-class score'] >= 1.2] #over 40% of the population in the selected neighbourhood are within middle income class
desireN = desireN[desireN['Single score'] >= 1.2] #over 40% of the population in the selected neighbourhood are one household
desireN = desireN[desireN['Without children score'] >= 1.2] #over 40% of the population in the selected neighbourhood are without children
desireN = desireN[desireN['Total Score'] >= sum_score.loc['mean']['Total Score']]
desireN.reset_index(drop=True, inplace=True)
desireN#.tail()

## C10. Journey to work
Based on mode to work, we could decide the opening hour of the restaurant.
Reference:
 - https://data.edmonton.ca/Census/2016-Census-Population-by-Mode-of-Transportation-N/h7iv-tgcw 2016 Census - Population by Mode of Transportation (Neighbourhood/Ward)

In [ ]:
d_journey = pd.read_csv("https://data.edmonton.ca/api/views/h7iv-tgcw/rows.csv?accessType=DOWNLOAD")
d_journey.tail()

### C10.1 Clean data

In [ ]:
d_journey.rename(columns={'Neighbourhood Name':'Neighbourhood'}, inplace=True)
d_journey['Neighbourhood']=d_journey['Neighbourhood'].str.title()
d_journey.drop(['Ward','Neighbourhood Number'],axis=1, inplace=True)

d_journey['Neighbourhood']=d_marital['Neighbourhood'].replace({'Mcconachie Area':'McConachie Area','Macewan':'MacEwan','Glenridding Area':'Glenridding Heights','Central Mcdougall': 'Central McDougall'})
d_journey.sort_values(['Neighbourhood'], inplace=True ,ascending=True)

journey2016 = pd.merge(d_journey,
                desireN[['Neighbourhood']],
                 on='Neighbourhood')
journey2016.tail()

In [ ]:
dm1=journey2016.copy()
dm2=desireN.copy()
dm1=dm1[['Neighbourhood']]
dm2=dm2[['Neighbourhood']]

dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

In [ ]:
journey2016.set_index('Neighbourhood', inplace=True)
journey2016.tail()

In [ ]:
#check whether respondse% over 25% of the total population
check = pd.merge(journey2016,
                dN_2016n2[['Neighbourhood','Total']],
                 on='Neighbourhood')
check['% respond']=(check['Car/Truck/Van (as Driver)']+check['Car/Truck/Van (as Passenger)']+check['Public Transit']+check['Walk']+check['Bicycle']+check['Other'])/check['Total']


### Since the data from city is not complete, import data from Federal census 2016

Reference：
 - https://public.tableau.com/profile/city.of.edmonton#!/vizhome/NeighbourhoodProfiles_FederalCensus2016/FamilyandLanguage
1. Download Tableau from the website
2. Access data by clicking 'data' from tableau and explore all into csv
3. upload to IBM_Cloud→Db2-vs→Manage→Open console→Load data (under GLN55437)

### Import new set of data

###### M1c.1 Connect to the database from SQL

Import data from SQL SERVER

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from JOURNEY_WORK"

#retrieve the query results into a pandas dataframe
f_jour = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
f_jour.head(8)

### Clean data (Federal census)

In [ ]:
f_jour.rename(columns={'NAME':'Neighbourhood','Mode_Commuting_Percentage':'Mode Commuting (%)'}, inplace=True)
f_jour['Neighbourhood']=f_jour['Neighbourhood'].str.title()
f_jour.drop(['Flag','Mode_Commuting_POP','Nbhd_Num','Number_of_Records'],axis=1, inplace=True)
f_jour['Neighbourhood']=f_jour['Neighbourhood'].replace({'Macewan':'MacEwan','Central Mcdougall': 'Central McDougall','Mcconachie Area':'McConachie Area'})
f_jour.sort_values(['Neighbourhood'], inplace=True ,ascending=True)


jour2016 = pd.merge(f_jour,
                      desireN[['Neighbourhood']],
                      on='Neighbourhood')
jour2016.head(6)

In [ ]:
dm1=jour2016.copy()
dm2=desireN.copy()
dm1=dm1.groupby('Neighbourhood',axis=0)['Mode Commuting (%)'].sum().reset_index()
dm1=dm1[['Neighbourhood']]
dm2=dm2[['Neighbourhood']]
dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

In [ ]:
S_jour2016=jour2016.copy()
S_jour2016=S_jour2016.pivot(index='Neighbourhood',columns='Commuing_Modes',values='Mode Commuting (%)')

S_jour2016=S_jour2016.astype(float)
S_jour2016.rename(columns={'Bicycle':'Bicycle (%)',
                       'Car, truck, van - as a driver':'Car, truck, van - as a driver (%)',
                       'Car, truck, van - as a passenger':'Car, truck, van - as a passenger (%)',
                       'Other method':'Other method (%)',
                       'Public transit':'Public transit (%)',
                       'Walked':'Walked (%)'}, inplace=True)

S_jour2016['Walked/Bicycle (%)']=S_jour2016['Walked (%)']+S_jour2016['Bicycle (%)']
S_jour2016['Car, truck, van (%)']=S_jour2016['Car, truck, van - as a driver (%)']+S_jour2016['Car, truck, van - as a passenger (%)']
S_jour2016['Other method (%)']=S_jour2016['Other method (%)']

S_jour2016.sort_values(["Car, truck, van - as a driver (%)"], inplace=True ,ascending=True)
columns_titles = ['Car, truck, van - as a driver (%)','Car, truck, van - as a passenger (%)','Public transit (%)','Bicycle (%)','Walked (%)','Walked/Bicycle (%)','Car, truck, van (%)','Other method (%)']
S_jour2016=S_jour2016.reindex(columns=columns_titles)
S_jour2016

In [ ]:
a=S_jour2016.copy()
a=a['Walked/Bicycle (%)'].reset_index()
d=S_jour2016.copy()
d=d['Public transit (%)'].reset_index()
b=S_jour2016.copy()
b=b['Car, truck, van (%)'].reset_index()
c=S_jour2016.copy()
c=c['Other method (%)'].reset_index()
jouney2016 = pd.merge(a,
                d,
                 on='Neighbourhood')
jouney2016 = pd.merge(jouney2016,
                b,
                 on='Neighbourhood')
jouney2016 = pd.merge(jouney2016,
                c,
                 on='Neighbourhood')
jouney2016.set_index('Neighbourhood', inplace=True)
jouney2016.tail()

In [ ]:
color_list = ['khaki','sandybrown','steelblue','lightseagreen','lightpink']
ax = jouney2016.plot(kind = 'bar',figsize = (20, 8), width = 0.8, color = color_list, edgecolor = None)


ax.set_title("2016 Mode Commuting to Work ", fontsize = 16)#use font size 16 for the title
ax.legend(fontsize='14',facecolor='white')#use font size 14 for legend
ax.tick_params(axis='x', labelsize=14)#use font size 14 for axis-lable
#ax.set_xlabel('') 

#Putting the percentage values on top of each bar
for p in ax.patches:
    ax.annotate(str(p.get_height().round(1)), (p.get_x() * 1.005, p.get_height() * 1.005), fontsize = 14) #use font size 14 for percentages
    
# remove the left, top, and right borders; make sure the color of x axix labels is black
ax.set_facecolor('xkcd:white') # change background to white  #ax.set_facecolor('xkcd:white')
ax.set_yticklabels([]) # turn off y ticks
ax.tick_params(left=False)
ax.axhline(0, color='black') # draw an x axix line 
ax.tick_params(axis='x', colors='black') # make sure the color of x axix labels is black   

ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

In [ ]:
S_jour2016=jour2016.copy()
S_jour2016=S_jour2016.pivot(index='Neighbourhood',columns='Commuing_Modes',values='Mode Commuting (%)')

S_jour2016=S_jour2016.astype(float)
S_jour2016.rename(columns={'Bicycle':'Bicycle (%)',
                       'Car, truck, van - as a driver':'Car, truck, van - as a driver (%)',
                       'Car, truck, van - as a passenger':'Car, truck, van - as a passenger (%)',
                       'Other method':'Other method (%)',
                       'Public transit':'Public transit (%)',
                       'Walked':'Walked (%)'}, inplace=True)

S_jour2016.sort_values(["Car, truck, van - as a driver (%)"], inplace=True ,ascending=True)
columns_titles = ['Car, truck, van - as a driver (%)','Car, truck, van - as a passenger (%)','Public transit (%)','Bicycle (%)','Walked (%)',]
S_jour2016=S_jour2016.reindex(columns=columns_titles)
S_jour2016

In [ ]:
color_list = ['khaki','sandybrown','steelblue','lightseagreen','lightpink']
ax = S_jour2016.plot(kind = 'bar',figsize = (20, 8), width = 0.8, color = color_list, edgecolor = None)


ax.set_title("2016 Mode Commuting to Work ", fontsize = 16)#use font size 16 for the title
ax.legend(fontsize='14',facecolor='white')#use font size 14 for legend
ax.tick_params(axis='x', labelsize=14)#use font size 14 for axis-lable
#ax.set_xlabel('') 

#Putting the percentage values on top of each bar
for p in ax.patches:
    ax.annotate(str(p.get_height().round(1)), (p.get_x() * 1.005, p.get_height() * 1.005), fontsize = 14) #use font size 14 for percentages
    
# remove the left, top, and right borders; make sure the color of x axix labels is black
ax.set_facecolor('xkcd:white') # change background to white  #ax.set_facecolor('xkcd:white')
ax.set_yticklabels([]) # turn off y ticks
ax.tick_params(left=False)
ax.axhline(0, color='black') # draw an x axix line 
ax.tick_params(axis='x', colors='black') # make sure the color of x axix labels is black   

ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

## C12. Immigration, language

### C12a.Citizenship

###### M1c.1 Connect to the database from SQL

Import data from SQL SERVER

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from CITIZENSHIP_2016"

#retrieve the query results into a pandas dataframe
f_citiz = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
f_citiz.head()

### Clean data (Federal census)

In [ ]:
f_citiz.rename(columns={'Name':'Neighbourhood','Citizenship_PT':"Non-Citizenship over the total neighbourhood's population(%)"}, inplace=True)
f_citiz['Neighbourhood']=f_citiz['Neighbourhood'].str.title()
f_citiz.drop(['COE_Citizenship_PT','COE_Citizenship_POP','Calculation1','Citizenship_POP','Nbhd_Num','Number_of_Records','Total_Citizenship'],axis=1, inplace=True)
f_citiz['Neighbourhood']=f_citiz['Neighbourhood'].replace({'Macewan':'MacEwan','Central Mcdougall': 'Central McDougall','Mcconachie Area':'McConachie Area'})
f_citiz.sort_values(['Neighbourhood'], inplace=True ,ascending=True)


citiz2016 = pd.merge(f_citiz,
                      desireN[['Neighbourhood']],
                      on='Neighbourhood')
citiz2016=citiz2016[~citiz2016.Citizenship_Status.str.contains("18")]
citiz2016.reset_index(drop=True, inplace=True)
citiz2016

In [ ]:
dm1=citiz2016.copy()
dm2=desireN.copy()
dm1=dm1[['Neighbourhood']]
dm2=dm2[['Neighbourhood']]
dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

In [ ]:
citiz2016=citiz2016.pivot(index='Neighbourhood',columns='Citizenship_Status',values="Non-Citizenship over the total neighbourhood's population(%)")
citiz2016=citiz2016.astype(float)
citiz2016.sort_values(["Not Canadian citizens"], inplace=True ,ascending=True)
columns_titles = ['Not Canadian citizens','Canadian citizens']
citiz2016=citiz2016.reindex(columns=columns_titles)
citiz2016

In [ ]:
color_list = ['yellowgreen','green']

ax = citiz2016.plot.barh(stacked=True,figsize = (15, 5), width = 0.8, color = color_list, align='center', edgecolor = None);



ax.set_title("2016 Non-Citizenship", fontsize = 11)#use font size 16 for the title
ax.legend(fontsize='10',facecolor='white',loc="lower right")#use font size 14 for legend
ax.tick_params(axis='x', labelsize=14)#use font size 14 for axis-lable
ax.set_xlabel("Percentage of Non-Citizenship over the total neighbourhood's population") 
ax.spines['left'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(str(width.round(1))+'%', xy=(left+width/2, bottom+height/2), 
                ha='center', va='center')
    
#ax.set_xlim(0, 35)
    
# remove the left, top, and right borders; make sure the color of x axix labels is black
ax.set_facecolor('xkcd:white') # change background to white  #ax.set_facecolor('xkcd:white')
#ax.set_xticklabels([]) # turn off y ticks
ax.tick_params(left=False)
ax.axvline(13.1, color='black',linestyle='dashed', alpha=0.5) # draw an x axix line 
ax.text(10, y=0, s='Non-Citizenship over the total citywise population is 13.1%', alpha=0.7, color='white')

ax.tick_params(axis='x', colors='black') # make sure the color of x axix labels is black    


### C12b. Place of Birth

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from PLACEBIRTH_2016"

#retrieve the query results into a pandas dataframe
f_PlBir = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
f_PlBir.head()

### Clean data (Federal census)

In [ ]:
f_PlBir.rename(columns={'Name':'Neighbourhood','Pob_Pt':"Neighbourhood's population percentage"}, inplace=True)
f_PlBir['Neighbourhood']=f_PlBir['Neighbourhood'].str.title()
f_PlBir.drop(['COE_Immigr_POP','Coe_Pob_Pop','Coe_Pob_Pt','ZeroAxis','Number_of_Records','Pob_Pop','Total_Immigrants'],axis=1, inplace=True)
f_PlBir['Neighbourhood']=f_PlBir['Neighbourhood'].replace({'Macewan':'MacEwan','Central Mcdougall': 'Central McDougall','Mcconachie Area':'McConachie Area'})
f_PlBir.sort_values(['Neighbourhood'], inplace=True ,ascending=True)


PlBir2016 = pd.merge(f_PlBir,
                      desireN[['Neighbourhood']],
                      on='Neighbourhood')

PlBir2016.reset_index(drop=True, inplace=True)
PlBir2016.head()

In [ ]:
dm1=PlBir2016.copy()
dm2=desireN.copy()
dm1=dm1[['Neighbourhood']]
dm2=dm2[['Neighbourhood']]
dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

In [ ]:
PlBir2016=PlBir2016.pivot(index='Neighbourhood',columns='Place_Of_Birth',values="Neighbourhood's population percentage")
PlBir2016=PlBir2016.astype(float)
PlBir2016.sort_values(["Asia"], inplace=True ,ascending=True)
columns_titles = ['Asia','Europe','Africa','Americas','Oceania and other places of birth']
PlBir2016=PlBir2016.reindex(columns=columns_titles)
PlBir2016

In [ ]:
color_list = ['khaki','sandybrown','steelblue','lightseagreen','lightpink']
ax = PlBir2016.plot(kind = 'bar',figsize = (20, 8), width = 0.8, color = color_list, edgecolor = None)


ax.set_title("2016 Immigrant Population by Place of Birth ", fontsize = 16)#use font size 16 for the title
ax.legend(fontsize='14',facecolor='white')#use font size 14 for legend
ax.tick_params(axis='x', labelsize=14)#use font size 14 for axis-lable
#ax.set_xlabel('') 

#Putting the percentage values on top of each bar
for p in ax.patches:
    ax.annotate(str(p.get_height().round(1))+'%', (p.get_x() * 1.005, p.get_height() * 1.005), fontsize = 14) #use font size 14 for percentages
    
# remove the left, top, and right borders; make sure the color of x axix labels is black
ax.set_facecolor('xkcd:white') # change background to white  #ax.set_facecolor('xkcd:white')
ax.set_yticklabels([]) # turn off y ticks
ax.tick_params(left=False)
ax.axhline(0, color='black') # draw an x axix line 
ax.tick_params(axis='x', colors='black') # make sure the color of x axix labels is black   

ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

### C12d. Language

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from LANGUAGE_2016"

#retrieve the query results into a pandas dataframe
f_langu = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
f_langu.head()

### Clean data (Federal census)

In [ ]:
f_langu.rename(columns={'NAME':'Neighbourhood','MotherTong_ForTop5_Percentage':"MotherTong_Top5 (%)"}, inplace=True)
f_langu['Neighbourhood']=f_langu['Neighbourhood'].str.title()
f_langu.drop(['MotherTong_ForTop5_POP','Nbhd_Num'],axis=1, inplace=True)
f_langu['Neighbourhood']=f_langu['Neighbourhood'].replace({'Macewan':'MacEwan','Central Mcdougall': 'Central McDougall','Mcconachie Area':'McConachie Area'})
f_langu.sort_values(['Neighbourhood'], inplace=True ,ascending=True)


language2016 = pd.merge(f_langu,
                      desireN[['Neighbourhood']],
                      on='Neighbourhood')

language2016.reset_index(drop=True, inplace=True)
language2016.head()

In [ ]:
dm1=language2016.copy()
dm2=desireN.copy()
dm1=dm1[['Neighbourhood']]
dm2=dm2[['Neighbourhood']]
dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

In [ ]:
lang2016=language2016.copy()
lang2016=lang2016.pivot(index='Neighbourhood',columns='Mother_Tongue_Language',values="MotherTong_Top5 (%)")
lang2016=lang2016.astype(float)
columns_titles = ['English','Mandarin','Cantonese','Tagalog (Pilipino, Filipino)','Korean','Spanish','French','German','Persian (Farsi)']
lang2016=lang2016.reindex(columns=columns_titles)
lang2016=lang2016.fillna(0)
lang2016.sort_values(["English"], inplace=True ,ascending=True)
lang2016

In [ ]:
#fig, ax = plt.subplots()

fig, ax = plt.subplots(figsize=(40,15))
ax = sns.heatmap(lang2016,vmin=0, vmax=10,annot = lang2016,fmt = '',linewidths=1, linecolor='white',cbar=False,cmap="RdYlGn",)

#sns.set(font_scale=2)
ax.set_xticklabels(ax.get_xticklabels(), rotation=30,fontsize = 25)
ax.set_yticklabels(ax.get_yticklabels(), rotation=30,fontsize = 25)

plt.xlabel("The frequency of occurrence of each category (%)",fontsize = 25)
#sns.heatmap(z, annot=False)

In [ ]:
language2016=language2016.pivot(index='Neighbourhood',columns='Mother_Tongue_Language',values="MotherTong_Top5 (%)")
language2016=language2016.astype(float)
columns_titles = ['Mandarin','Cantonese','Tagalog (Pilipino, Filipino)','Korean','Spanish','French','German','Persian (Farsi)']
language2016=language2016.reindex(columns=columns_titles)
language2016=language2016.fillna(0)
language2016.sort_values(["Mandarin"], inplace=True ,ascending=True)
language2016

In [ ]:
color_list = ['khaki','khaki','steelblue','lightseagreen','sandybrown','darkorange','chocolate','lightpink']

ax = language2016.plot.barh(stacked=True,figsize = (15, 5), width = 0.8, color = color_list, align='center', edgecolor = None);



ax.set_title("2016 Non-Citizenship", fontsize = 11)#use font size 16 for the title
ax.legend(fontsize='10',facecolor='white',loc="lower right")#use font size 14 for legend
ax.tick_params(axis='x', labelsize=14)#use font size 14 for axis-lable
ax.set_xlabel("Percentage of Non-Citizenship over the total neighbourhood's population") 
ax.spines['left'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(str(width.round(1))+'%', xy=(left+width/2, bottom+height/2), 
                ha='center', va='center')
    
#ax.set_xlim(0, 35)
    
# remove the left, top, and right borders; make sure the color of x axix labels is black
ax.set_facecolor('xkcd:white') # change background to white  #ax.set_facecolor('xkcd:white')
#ax.set_xticklabels([]) # turn off y ticks
ax.tick_params(left=False)
ax.axvline(13.1, color='black',linestyle='dashed', alpha=0.5) # draw an x axix line 
ax.text(10, y=0, s='Non-Citizenship over the total citywise population is 13.1%', alpha=0.7, color='white')

ax.tick_params(axis='x', colors='black') # make sure the color of x axix labels is black    


In [ ]:
color_list = ['khaki','sandybrown','steelblue','lightseagreen','lightpink']
ax = language2016.plot(kind = 'bar',figsize = (20, 8), width = 0.8, color = color_list, edgecolor = None)


ax.set_title("2016 Immigrant Population by Place of Birth ", fontsize = 16)#use font size 16 for the title
ax.legend(fontsize='14',facecolor='white')#use font size 14 for legend
ax.tick_params(axis='x', labelsize=14)#use font size 14 for axis-lable
#ax.set_xlabel('') 

#Putting the percentage values on top of each bar
for p in ax.patches:
    ax.annotate(str(p.get_height().round(1))+'%', (p.get_x() * 1.005, p.get_height() * 1.005), fontsize = 14) #use font size 14 for percentages
    
# remove the left, top, and right borders; make sure the color of x axix labels is black
ax.set_facecolor('xkcd:white') # change background to white  #ax.set_facecolor('xkcd:white')
ax.set_yticklabels([]) # turn off y ticks
ax.tick_params(left=False)
ax.axhline(0, color='black') # draw an x axix line 
ax.tick_params(axis='x', colors='black') # make sure the color of x axix labels is black   

ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

## Ca. Bussiness revitalization zone
Reference:    
Note: The name BRZ was changed to BIA on July 1, 2016
 - https://tram.mcgill.ca/Teaching/srp/documents/Ilja.pdf General idea what is BIA
 - https://www.edmonton.ca/business_economy/business_resources/business-improvement-areas.aspx Edmonton BIA info
 - https://www.edmonton.ca/business_economy/business_resources/business-improvement-area-contacts.aspx detail of wach BIA zone
 - https://www.edmonton.ca/business_economy/documents/BIAmap2018.pdf 2018 BIA ZONE
 - https://www.edmonton.ca/city_government/documents/PDF/04110_Business_Revitalization_Zone_Area.pdf 2015 BRZ ZONE

### Ca.1 HEAT MAP - BIA zone

1. Method to creat your own GeoJson file
 - http://blog.infographics.tw/2016/01/draw-map-with-geojson-io/
2. After obtaining all geometries, save as GeoJson, then upload the file on Github
3. Method to read GeoJson files create from http://geojson.io/    
 - https://stackoverflow.com/questions/60219180/importing-json-file-from-github-into-python-getting-error-jsondecodeerror-exp
 
 Reference:
 - https://data.edmonton.ca/Sustainable-Development/Business-Improvement-Areas-Business-Licenses/eqs3-bp4i
 - https://data.edmonton.ca/Geospatial-Boundaries/Business-Improvement-Areas/34cp-nvx7

In [ ]:
import geopandas as gpd
#response = requests.get(r"https://raw.githubusercontent.com/cgao2/BIA-map-/master/BIA_MAP.geojson") #create by me
response = requests.get(r"https://data.edmonton.ca/api/geospatial/34cp-nvx7?method=export&format=GeoJSON") #provde by city of Edmonton

data = response.json()
BIA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

BIA#.head()

In [ ]:
#dataward = {'name':  ['124 Street and Area BIA', 'Alberta Avenue BIA', 'Beverly BIA','Chinatown and Area BIA', 'Downtown BIA','Fort Road and Area BIA', 'French Quarter and Area BIA','Kingsway BIA', 'North Edge BIA','Northwest Industrial BIA', 'Old Strathcona BIA','Stony Plain Road and Area BIA','The Crossroads BIA'],
#        'BIA_ID': [1,2,3,4,5,6,7,8,9,10,11,12,13],
#        }
dataward = {'name':  ['124 Street and Area', 'Alberta Avenue', 'Beverly','Chinatown and Area', 'Downtown','Fort Road and Area', 'French Quarter and Area','Kingsway', 'North Edge','Northwest Industrial', 'Old Strathcona','Stony Plain Road and Area','The Crossroads'],
        'BIA_ID': list(range(1,len(BIA)+1))
        }

df_BIA = pd.DataFrame (dataward, columns = ['name','BIA_ID'])
df_BIA#.head()

In [ ]:
BIAmerge = pd.merge(BIA,
                df_BIA,
                 on='name')
BIAmerge

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisA.tail()

In [ ]:
NeisA.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisA,
                #dtopNei,
                     desireN[['Neighbourhood']],
                 on='Neighbourhood')
NeiAmerge.drop(['area_km2','number'],axis=1, inplace=True)
NeiAmerge.rename(columns={'Neighbourhood':'name'}, inplace=True)
NeiAmerge['BIA_ID']=[1]*len(NeiAmerge)
NeiAmerge.tail()

NeiBIAmerge = NeiAmerge.append(BIAmerge, ignore_index=True)

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09
colormap = cm.linear.Paired_12.scale(BIAmerge['BIA_ID'].min(), BIAmerge['BIA_ID'].max()).to_step(BIAmerge['BIA_ID'].max()) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edm_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron') #, tiles = 'Stamen Toner')


style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['BIA_ID']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



# tooltip=folium.features.GeoJsonTooltip(fields=['Ward','WardID'],
#            aliases=['Ward','Ward number'],
#            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
#            sticky=True
#        )


g = folium.GeoJson(
    BIAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name','BIA_ID'],
        aliases=['name','BIA ID'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edm_map)
folium.GeoJson(
    NeiAmerge,
    style_function = lambda x: {
        'color': 'black',
        'weight': 2.5,
        'fillOpacity': 0
    }).add_to(edm_map)
colormap.add_to(edm_map)

#edm_map

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Business Improvement Area (BIA)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#a6cee3;opacity:0.7;'></span>124 Street Area</li>
    <li><span style='background:#1f78b4;opacity:0.7;'></span>Alberta Avenue</li>
    <li><span style='background:#b2df8a;opacity:0.7;'></span>Beverly</li>
    <li><span style='background:#33a02c;opacity:0.7;'></span>Chinatown Area</li>
    
    <li><span style='background:#dfc27d;opacity:0.7;'></span>Downtown</li>
    <li><span style='background:tomato;opacity:0.7;'></span>Fort Road Area</li>
    <li><span style='background:chocolate;opacity:0.7;'></span>French Quarter Area</li>
    <li><span style='background:orange;opacity:0.7;'></span>Kingsway</li>
    <li><span style='background:darkorange;opacity:0.7;'></span>North Edge</li>
    
    <li><span style='background:#cab2d6;opacity:0.7;'></span>Northwest Industrial</li>
    <li><span style='background:#6a3d9a;opacity:0.7;'></span>Old Strathcona</li>
    <li><span style='background:#ffff99;opacity:0.7;'></span>Stony Plain Road Area</li>
    <li><span style='background:#b15928;opacity:0.7;'></span>The Crossroads</li>
    


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edm_map.get_root().add_child(macro)

edm_map

In [ ]:
scoreBIA = {'Neighbourhood':  ['Westmount','Oliver','Downtown','Queen Alexandra','Strathcona','Paisley','Empire Park'],
        'BIA score': [1,1,1,1,1,0,0]
        }

dw_BIA = pd.DataFrame (scoreBIA, columns = ['Neighbourhood','BIA score'])
dw_BIA

## Cb. Bussiness zoning bylaw

### Cb.1 Visualized resturant/special food permit in residential zones and commercial zones
1. Most Residential Zones
2. All Industrial Zones
3. All Urban Service Zones
4. All Agricultural and Reserve Zones      

**are not counted in the following discussion**  

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/b4f2-gf2b?method=export&format=GeoJSON") #provde by city of Edmonton

data = response.json()
BIA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

BIA.head()

In [ ]:
a=BIA[BIA.descript.str.contains("Commercial")]
a=a.append(BIA[BIA.descript.str.contains("Business")], ignore_index=True)
a=a.append(BIA[BIA.descript.str.contains("Corridor")], ignore_index=True)
a=a.append(BIA[BIA.descript.str.contains("Shopping")], ignore_index=True)
a=a.append(BIA[BIA.zoning.str.contains("UW")], ignore_index=True)
a=a.append(BIA[BIA.descript.str.contains("Griesbach Village Centre Zone")], ignore_index=True)
a=a.append(BIA[BIA.descript.str.contains("Heritage Area Zone")], ignore_index=True)
a=a.append(BIA[BIA.descript.str.contains("Marquis")], ignore_index=True)
a=a.append(BIA[BIA.descript.str.contains("Mixed Use Zone")], ignore_index=True)
a=a.append(BIA[BIA.descript.str.contains("Arena and Entertainment District Zone")], ignore_index=True)

a=a[~a.descript.str.contains("Industrial")]
a=a[~a.zoning.str.contains("MA3")]
#a['id']=a['id'].astype(int)

a['ID']=[1]*len(a)
#a = pd.merge(a, b[['zoning','ID']],on='zoning')
a.tail()

In [ ]:
a1=BIA[BIA.zoning.str.contains("DC1")]
a1=a1.append(BIA[BIA.zoning.str.contains("DC2")], ignore_index=True)

a1['ID']=[2]*len(a1)
a1.tail()

In [ ]:
b=BIA[BIA.zoning.str.contains("RA7")]
b=b.append(BIA[BIA.zoning.str.contains("RA8")], ignore_index=True)
b=b.append(BIA[BIA.zoning.str.contains("RA9")], ignore_index=True)
b=b.append(BIA[BIA.zoning.str.contains("BLMR")], ignore_index=True)
b=b.append(BIA[BIA.zoning.str.contains("CCHD")], ignore_index=True)
b=b.append(BIA[BIA.zoning.str.contains("CCMD")], ignore_index=True)
b=b[~b.descript.str.contains("Ambleside Low Rise Apartment Zone")]


b['ID']=[3]*len(b)
b.tail()

In [ ]:
c=a.append([a1,b], ignore_index=True)

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisA.tail()

In [ ]:
NeisA.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisA,
                     desireN[['Neighbourhood']],
                 on='Neighbourhood')
NeiAmerge.drop(['area_km2','number'],axis=1, inplace=True)
NeiAmerge.rename(columns={'Neighbourhood':'descript'}, inplace=True)
NeiAmerge['ID']=[4]*len(NeiAmerge)
NeiAmerge['zoning']=['Neighbourhood']*len(NeiAmerge)
NeiAmerge

In [ ]:
NeiBIAmerge = c.append(NeiAmerge, ignore_index=True)
NeiBIAmerge.tail(10)

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09
colormap = cm.linear.Paired_09.scale(0, NeiBIAmerge['ID'].max()).to_step(4) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edm_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron')


style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['ID']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



# tooltip=folium.features.GeoJsonTooltip(fields=['Ward','WardID'],
#            aliases=['Ward','Ward number'],
#            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
#            sticky=True
#        )


g = folium.GeoJson(
    NeiBIAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['zoning','descript','ID'],
        aliases=['zoning','descript','ID'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edm_map)

colormap.add_to(edm_map)

edm_map

## C11. Food license visualization
Reference:
 - https://data.edmonton.ca/Sustainable-Development/City-of-Edmonton-Business-Licenses/qhi4-bdpu/data

In [ ]:
d_license = pd.read_csv("https://data.edmonton.ca/api/views/qhi4-bdpu/rows.csv?accessType=DOWNLOAD")
d_license.tail()

### C11.1 Clean data

Check is there any Licence expired

In [ ]:
d_food=d_license.copy()
d_food['Expiry date'] = d_food['Expiry Date'].str.strip().str[6:10]+d_food['Expiry Date'].str.strip().str[0:2]+d_food['Expiry Date'].str.strip().str[3:5]
d_food['Expiry date'] =d_food['Expiry date'].astype(int)
d_food = d_food[d_food['Expiry date'] > 20200801] #enter current date (date before the current date means the license is expired, and the row will be droped)

a = d_food[d_food['Category'] == "Restaurant or Food Service"]
b = d_food[d_food['Category'] == "Food Processing"]

d_food=a.append(b, ignore_index=True)
d_food = d_food[d_food['Expiry Date'].str.strip().str[6:11] >= '2020'] #drop expiry date before year 2020

d_food.drop(['Ward','Licence Number','Count','Neighbourhood ID','Licence Status','Issue Date','Expiry Date','Expiry date'],axis=1, inplace=True)
d_food.reset_index(drop=True, inplace=True)

d_food.head()

rows in d_food dataframe without latitude, longitude, address information

In [ ]:
foodnull2019=d_food.copy()
foodnull2019 = foodnull2019[foodnull2019['Latitude'].isnull()]

rows in d_food dataframe with latitude, longitude, address information

In [ ]:
food2019=d_food.copy()
food2019 = food2019[food2019['Latitude'].notnull()]

In [ ]:
locations = food2019[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[4]

In [ ]:
from folium.plugins import MarkerCluster

In [ ]:
map = folium.Map(location=[53.535411, -113.507996], 
                 #name="Light Map", 
                 tiles='CartoDB positron',
                 zoom_start=12)
marker_cluster = MarkerCluster().add_to(map)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=food2019.iloc[point][1]).add_to(marker_cluster) 
map


In [ ]:
neighborhoods=d_food.copy()
neighborhoods=neighborhoods[['Neighbourhood','Latitude','Longitude']]
neighborhoods['Borough']=['Edmonton']*len(d_food)
neighborhoods.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
neighborhoods=neighborhoods.dropna()
# create map of Edmonton using latitude and longitude values
map_edmonton = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=0.1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_edmonton)  
    
map_edmonton

### Food service in selected neighbourhood

In [ ]:
Ndisired_food = pd.merge(d_food,
                        desireN[['Neighbourhood']],
                        on='Neighbourhood')
#Ndisired_food.drop(['Latitude','Longitude','Location'],axis=1, inplace=True)
Ndisired_food.sort_values(['Trade Name'], inplace=True ,ascending=True)
Ndisired_food.reset_index(drop=True, inplace=True)
#Ndisired_food.drop_duplicates(subset ="First Name", 
#                     keep = False, inplace = True) 
Ndisired_food.head()

In [ ]:
len(Ndisired_food)

In [ ]:
dulicatefood = Ndisired_food[Ndisired_food.duplicated(['Trade Name', 'Address'])]
dulicatefood.tail()

In [ ]:
Ndisired_food = Ndisired_food[~Ndisired_food.duplicated(['Trade Name', 'Address'])]
len(Ndisired_food)

### !!!!!!Store Ndisired_food: food license in selected Neighbouthood

In [ ]:
%store Ndisired_food

In [ ]:
Ndisired_food.to_csv("Restaurant.csv",index=False)

In [ ]:
C_NeiFood=Ndisired_food.groupby('Neighbourhood',axis=0)['Trade Name'].count().reset_index()
C_NeiFood

### Clean crape data from Yelp.ca and Zomato.com

In [ ]:
SelectNei_Res=pd.read_csv("SelectNei_restaurant.csv")
SelectNei_Res.head()

In [ ]:
SelectNei_Restype=SelectNei_Res.groupby('Type',axis=0)['Trade Name'].count().reset_index()
SelectNei_Restype

### Casual dinings with/without wine/liquior service in selected neighbourhoods

In [ ]:
SelectNei_OpenRes=SelectNei_Res[SelectNei_Res.Type.str.contains("Dining")]
SelectNei_OpenRes=SelectNei_OpenRes.append(SelectNei_Res[SelectNei_Res.Type.str.contains("Pub")], ignore_index=True)
SelectNei_OpenRes=SelectNei_OpenRes.append(SelectNei_Res[SelectNei_Res.Type.str.contains("Bar")], ignore_index=True)

In [ ]:
SelectNei_Restype=SelectNei_OpenRes.groupby('Type',axis=0)['Trade Name'].count().reset_index()
SelectNei_Restype

In [ ]:
SelectNei_Restype=SelectNei_OpenRes.groupby('Neighbourhood',axis=0)['Trade Name'].count().reset_index()
SelectNei_Restype

In [ ]:
SelectNei_OpenRes['category']=SelectNei_OpenRes['cuisines'].str.title()+','+SelectNei_OpenRes['cuisines.1'].str.title()+','
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].astype(str)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace("New","",case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('\\(|\\)',"")
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('Newcanadian','Canadian',case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace("Traditional","",case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('Asian Fusion','Asian',case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('Ramen','Japanese',case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('Noodle','Chinese',case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('Creperies','French',case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('Crepes','French',case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('Cantonese','Chinese',case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('Chineses','Chinese',case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace(',0','',case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('Modern European','European',case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('Cajun/Creole','Cajun',case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('Barbeque','BBQ',case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('Fish & Chip','British',case=False)
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('\\?',"")
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.replace('s,',",")
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].str.title()
SelectNei_OpenRes.head()

In [ ]:
SelectNei_OpenRes.shape

### keyword of resturant

In [ ]:
SelectNei_OpenRes['category']=SelectNei_OpenRes['category'].astype(str)
a=SelectNei_OpenRes.iloc[3][12]
print(a)
count = a.count(',')
print(count)
a=a.split(",")
print(a[0])
b=a[0].strip()
#b=b.title()
print(b)

In [ ]:
keyword=[]
for i in range(len(SelectNei_OpenRes)):
    colmn=SelectNei_OpenRes.shape[1]-1
    a=SelectNei_OpenRes.iloc[i][colmn]
    word=a.split(",")
    count = a.count(',')
    for j in range((count)):
        b=word[j]
        c=b.strip()
        #c=c.title()
        keyword.append(c)

In [ ]:
from collections import Counter
counts = Counter(keyword)
counts=pd.DataFrame(counts.items())
counts.sort_values([0], inplace=True ,ascending=True)
counts.reset_index(drop=True, inplace=True)
counts

### Catergory by country

In [ ]:
country=["African","American","Asian","Australian","Brazilian","British","Cajun","Canadian","Caribbean","Chinese","Czech",
        "Ethiopian","European","Filipino","French","German","Greek","Hala","Hawaiian","Indian","Irish","Italian",
        "Jamaican","Japanese","Korean","Latin American","Laotian","Lebanese","Mediterranean","Mexican","Middle Eastern",
        "Pakistani","Portuguese","Southern","Spanish","Thai","Turkish","Tex-Mex","Vietnamese"]
country[0]

In [ ]:
SelectNei_Rescountry=SelectNei_OpenRes[SelectNei_OpenRes.category.str.contains('test')]
SelectNei_Rescountry

In [ ]:
SelectNei_Rescountry=SelectNei_OpenRes[SelectNei_OpenRes.category.str.contains('test')]
rec_country=[]
for i in range(len(country)):
    region=country[i]
    a=SelectNei_OpenRes[SelectNei_OpenRes.category.str.contains(region)]
    SelectNei_Rescountry=SelectNei_Rescountry.append(a, ignore_index=True)
    b=[region]*len(a)
    rec_country.extend(b)
#print(rec_country)
SelectNei_Rescountry['region']=rec_country
SelectNei_Rescountry.tail()

In [ ]:
len(SelectNei_Rescountry)

In [ ]:
dm1=SelectNei_Rescountry.copy()
dm2=SelectNei_OpenRes.copy()
dm1.drop(['region'],axis=1, inplace=True)
dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]
dmiss.reset_index(drop=True, inplace=True)
dmiss['region']=['American']*len(dmiss)
dmiss.tail()

In [ ]:
restotal=SelectNei_Rescountry.append(dmiss, ignore_index=True)
Summary_region=restotal.groupby(['Neighbourhood','region'],axis=0)['Trade Name'].count().reset_index()

Summary_region=Summary_region.pivot(index='Neighbourhood',columns='region',values='Trade Name')
#Summary_region.sort_values(['Trade Name'], inplace=True ,ascending=True)
Summary_region=Summary_region.fillna(0)
Summary_region=Summary_region.astype(int)

Summary_region

In [ ]:
Summary_region=Summary_region/Summary_region.sum(axis=1)[:,None]*100
Summary_region=Summary_region.round(0)
Summary_region

In [ ]:
#fig, ax = plt.subplots()

fig, ax = plt.subplots(figsize=(40,8))
ax = sns.heatmap(Summary_region,vmin=0, vmax=30,annot = Summary_region,fmt = '',linewidths=1, linecolor='white',cbar=False,cmap="YlOrBr",)
#heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=30) 
#heatmap.set_yticklabels(heatmap.get_yticklabels(), rotation=90)
sns.set(font_scale=1)
ax.set_xticklabels(ax.get_xticklabels(), rotation=30,fontsize = 18)
ax.set_yticklabels(ax.get_yticklabels(), rotation=30,fontsize = 18)

plt.xlabel("The frequency of occurrence of each category (%)",fontsize = 18)
#sns.heatmap(z, annot=False)

## **Appendix**

## C.A.1 Import Marital status    

#### Year 2016

###### Import data from SQL SERVER

#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from MARITAL_2016"

#retrieve the query results into a pandas dataframe
d_marital = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
d_marital.tail()

###### Import data from City of Edmonton open data bank

In [ ]:
d_marital = pd.read_csv("https://data.edmonton.ca/api/views/fcvm-p6fa/rows.csv?accessType=DOWNLOAD")
d_marital.tail()

#### Year 2001    
Reference:
https://www.edmonton.ca/business_economy/documents/InfraPlan/HholdsizeIncome.pdf

###### Import data from SQL

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from HOUSEHOLD_SIZE2001"

#retrieve the query results into a pandas dataframe
d_mari2001 = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
d_mari2001.tail()

### C.A.1 Clean data 2001

In [ ]:
len(d_mari2001)

In [ ]:
f_mari2001=d_mari2001.copy()
f_mari2001.rename(columns={'Neighbourhoods':'Neighbourhood'}, inplace=True)
f_mari2001.drop(['Household_Size','Income_Average__','Income_Median__','Column_10'],axis=1, inplace=True)
f_mari2001.drop(f_mari2001.tail(1).index,inplace=True)
f_mari2001 = f_mari2001.astype(str)
import re
f_mari2001 = f_mari2001.applymap(lambda x: re.sub(r'^-$', str(0), x))

f_mari2001['Households']= f_mari2001['Households'].astype(float)
f_mari2001['1 person']= f_mari2001['1 person'].astype(float)
f_mari2001['2 persons']= f_mari2001['2 persons'].astype(float)
f_mari2001['3 persons']= f_mari2001['3 persons'].astype(float)
f_mari2001['4-5 persons']= f_mari2001['4-5 persons'].astype(float)
f_mari2001['6 or more']= f_mari2001['6 or more'].astype(float)

f_mari2001['1 person household (%)']=f_mari2001['1 person']/f_mari2001['Households']*100
f_mari2001['≥2 person household (%)']=(f_mari2001['2 persons']+f_mari2001['3 persons']+f_mari2001['4-5 persons']+f_mari2001['6 or more'])/f_mari2001['Households']*100

f_mari2001['1 person household (%)']=f_mari2001['1 person household (%)'].round(1)
f_mari2001.sort_values(['Neighbourhood'], inplace=True ,ascending=True)
f_mari2001.tail()

### HEAT MAP - 2001

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisB = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisB.tail()

In [ ]:
NeisB.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisB,
                f_mari2001[['Neighbourhood','1 person household (%)']],
                 on='Neighbourhood')

NeiAmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09
colormap = cm.linear.RdPu_08.scale(0, 60).to_step(6) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron')
colormap.caption = "Percentage of 1 person household by neighbourhood in 2016"

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['1 person household (%)']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Neighbourhood','1 person household (%)'],
        aliases=['Neighbourhood','1 person household (%)'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

colormap.add_to(edmA_map)
#edmA_map

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>1 person household (%)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#FEEBE2;opacity:0.7;'></span><10%</li>
    <li><span style='background:#FCC5C0;opacity:0.7;'></span>10% to 20%</li>
    <li><span style='background:#FA9FB5;opacity:0.7;'></span>20% to 30%</li>
    <li><span style='background:#F768A1;opacity:0.7;'></span>30% to 40%</li>
    <li><span style='background:#AE017E;opacity:0.7;'></span>40% to 50%</li>
    <li><span style='background:#7A0177;opacity:0.7;'></span>>50%</li>


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edmA_map.get_root().add_child(macro)

edmA_map

### C.A.1 Clean data 2016    
Reference:
https://data.edmonton.ca/Census/2016-Census-Population-by-Marital-Status-Neighbour/fcvm-p6fa/data

#### 2016 Stat Canada

d_marital.rename(columns={'NAME':'Neighbourhood'}, inplace=True)
d_marital['Neighbourhood']=d_marital['Neighbourhood'].str.title()
d_marital.drop(['1','Flag','Marital_Status_POP','Nbhd_Num'],axis=1, inplace=True)

d_marital['Neighbourhood']=d_marital['Neighbourhood'].replace({'Mccauley':'McCauley',
                                                             'Mckernan':'McKernan',
                                                             'Mcleod':'McLeod',
                                                             'Mcqueen':'McQueen',
                                                             'Place Larue':'Place LaRue',
                                                             'Rapperswil':'Rapperswill',
                                                             "River'S Edge":"River's Edge",
                                                             'Westbrook Estate':'Westbrook Estates',
                                                             'Mcconachie Area':'McConachie Area',
                                                             'University Of Alberta Farm':'University of Alberta Farm',
                                                             'University Of Alberta':'University of Alberta',
                                                             'Glenridding Area':'Glenridding Heights',
                                                             'Macewan':'MacEwan',
                                                             'Central Mcdougall': 'Central McDougall'})
d_marital=d_marital.groupby(['Neighbourhood','Marital_Status'],axis=0)['Marital_Status_Percentage'].sum().reset_index()
d_marital.sort_values(['Neighbourhood'], inplace=True ,ascending=True)

marital2016 = pd.merge(d_marital,
                dclassMED[['Neighbourhood']],
                 on='Neighbourhood')
marital2016.tail(6)

dm1=marital2016.copy()
dm2=dtopNei.copy()
dm1=dm1.groupby('Neighbourhood',axis=0)['Marital_Status_Percentage'].sum().reset_index()
dm1=dm1[['Neighbourhood']]
dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

S_mari2016=marital2016.copy()
S_mari2016=S_mari2016.pivot(index='Neighbourhood',columns='Marital_Status',values='Marital_Status_Percentage')

S_mari2016=S_mari2016.astype(float)
S_mari2016['1 person household (%)']=S_mari2016['Never married']
S_mari2016['≥2 person household (%)']=S_mari2016['Married']+S_mari2016['Living common law']
S_mari2016['other (%)']=S_mari2016['Divorced']+S_mari2016['Separated']+S_mari2016['Widowed']

S_mari2016.sort_values(['1 person household (%)'], inplace=True ,ascending=True)

S_mari2016.tail()

a=S_mari2016.copy()
a=a['1 person household (%)'].reset_index()
b=S_mari2016.copy()
b=b['≥2 person household (%)'].reset_index()
c=S_mari2016.copy()
c=c['other (%)'].reset_index()
marital2016c = pd.merge(a,
                b,
                 on='Neighbourhood')
marital2016c= pd.merge(marital2016c,
                c,
                 on='Neighbourhood')
marital2016c['1 person household (%)']=marital2016c['1 person household (%)'].astype(float).round(1)
#marital2016c.set_index('Neighbourhood', inplace=True)
marital2016c.tail()

#### 2016 City of Edmonton

In [ ]:
d_marital.rename(columns={'Neighbourhood Name':'Neighbourhood'}, inplace=True)
d_marital['Neighbourhood']=d_marital['Neighbourhood'].str.title()
d_marital.drop(['Ward','Neighbourhood Number'],axis=1, inplace=True)

d_marital['Neighbourhood']=d_marital['Neighbourhood'].replace({'Mccauley':'McCauley',
                                                             'Mckernan':'McKernan',
                                                             'Mcleod':'McLeod',
                                                             'Mcqueen':'McQueen',
                                                             'Place Larue':'Place LaRue',
                                                             'Rapperswil':'Rapperswill',
                                                             "River'S Edge":"River's Edge",
                                                             'Westbrook Estate':'Westbrook Estates',
                                                             'Mcconachie Area':'McConachie Area',
                                                             'University Of Alberta Farm':'University of Alberta Farm',
                                                             'University Of Alberta':'University of Alberta',
                                                             'Glenridding Area':'Glenridding Heights',
                                                             'Macewan':'MacEwan',
                                                             'Central Mcdougall': 'Central McDougall'})

d_marital.sort_values(['Neighbourhood'], inplace=True ,ascending=True)

marital2016 = pd.merge(d_marital,
                dclassMED[['Neighbourhood']],
                 on='Neighbourhood')
marital2016.tail(6)

In [ ]:
dm1=marital2016.copy()
dm2=dclassMED.copy()
dm1=dm1[['Neighbourhood']]
dm2=dm2[['Neighbourhood']]

dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss#.tail()

In [ ]:
marital2016c=marital2016.copy()
marital2016c.drop(['No Response'],axis=1, inplace=True)
cols_to_sum = marital2016c.columns[ 0: marital2016.shape[1]]
marital2016c['Total']=marital2016c[cols_to_sum].sum(axis=1)

marital2016c['Married (%)']=marital2016c['Married']/marital2016c['Total']*100
marital2016c['Common Law (%)']=marital2016c['Common Law']/marital2016c['Total']*100

marital2016c['Separated/Divorced (%)']=marital2016c['Separated/Divorced']/marital2016c['Total']*100
marital2016c['Widowed (%)']=marital2016c['Widowed']/marital2016c['Total']*100

marital2016c['1 person household (%)']=marital2016c['Never Married']/marital2016c['Total']*100
marital2016c['≥2 person household (%)']=marital2016c['Married (%)']+marital2016c['Common Law (%)']
marital2016c['other (%)']=marital2016c['Separated/Divorced (%)']+marital2016c['Widowed (%)']
marital2016c=marital2016c.fillna(0)
marital2016c=marital2016c.round(1)
marital2016c.drop(['Married','Common Law','Separated/Divorced','Widowed','Total','Never Married','Married (%)','Common Law (%)','Separated/Divorced (%)','Widowed (%)'],axis=1, inplace=True)
marital2016c.sort_values(['1 person household (%)'], inplace=True ,ascending=True)
marital2016c.tail()

### HEAT MAP - 2016

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisB = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisB.tail()

In [ ]:
NeisB.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisB,
                marital2016c[['Neighbourhood','1 person household (%)']],
                 on='Neighbourhood')
NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'Cavanagh', '1 person household (%)'] = 20.8 #reference: stat canada 2016 census
NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'University of Alberta Farm', '1 person household (%)'] = 40.3
NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'University of Alberta', '1 person household (%)'] = 88.6
NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'Place LaRue', '1 person household (%)'] = 7.1
NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'Crystalliana Nera East', '1 person household (%)'] =44 

NeiAmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09
colormap = cm.linear.RdPu_08.scale(0, 60).to_step(6) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron')
colormap.caption = "Percentage of 1 person household by neighbourhood in 2016"

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['1 person household (%)']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Neighbourhood','1 person household (%)'],
        aliases=['Neighbourhood','1 person household (%)'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

colormap.add_to(edmA_map)
#edmA_map

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>1 person household (%)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#FEEBE2;opacity:0.7;'></span><10%</li>
    <li><span style='background:#FCC5C0;opacity:0.7;'></span>10% to 20%</li>
    <li><span style='background:#FA9FB5;opacity:0.7;'></span>20% to 30%</li>
    <li><span style='background:#F768A1;opacity:0.7;'></span>30% to 40%</li>
    <li><span style='background:#AE017E;opacity:0.7;'></span>40% to 50%</li>
    <li><span style='background:#7A0177;opacity:0.7;'></span>>50%</li>


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edmA_map.get_root().add_child(macro)

edmA_map

### comparison 2001 and 2016

In [ ]:
f_marri2001=f_mari2001.copy()


f_marri2001['Neighbourhood']=f_marri2001['Neighbourhood'].replace({'Thorncliffe':'Thorncliff',
                                                                'Westbrook Estate':'Westbrook Estates',
                                                                'CFB Griesbach':'Griesbach',
                                                                 'Mcleod':'McLeod',
                                                                 'Millwoods Town Centre':'Mill Woods Town Centre',
                                                                 'Clareview Business Park':'Clareview Town Centre',
                                                                 'Place La Rue':'Place LaRue',
                                                                 'Eaux Claires - Belle Rive':'Eaux Claires',
                                                                 'Mayliewan - Ozerna':'Mayliewan',
                                                                 'Hollick Kenyon':'Hollick-Kenyon',
                                                                   'Rural West':'Rural West Big Lake',
                                                                   'Rural NE - Horse Hill':'Rural North East Horse Hill',
                                                                   'Rural NE - South Sturgeon':'Rural North East South Sturgeon'
                                                                })
f_marri2001=f_marri2001.append(f_mari2001[f_mari2001.Neighbourhood.str.contains("Eaux Claires - Belle Rive")], ignore_index=True)
f_marri2001['Neighbourhood']=f_marri2001['Neighbourhood'].replace({'Eaux Claires - Belle Rive':'Belle Rive'})
f_marri2001=f_marri2001.append(f_mari2001[f_mari2001.Neighbourhood.str.contains("Mayliewan - Ozerna")], ignore_index=True)
f_marri2001['Neighbourhood']=f_marri2001['Neighbourhood'].replace({'Mayliewan - Ozerna':'Ozerna'})
f_marri2001=f_marri2001.append(f_mari2001[f_mari2001.Neighbourhood.str.contains("Millwoods Town Centre")], ignore_index=True)
f_marri2001['Neighbourhood']=f_marri2001['Neighbourhood'].replace({'Millwoods Town Centre':'Mill Woods Park'})

single01and16 = pd.merge(marital2016c[['Neighbourhood','1 person household (%)']],
                f_marri2001[['Neighbourhood','1 person household (%)']],
                 on='Neighbourhood')
single01and16['1 person household percentage difference between 2001 to 2016 (%)']=single01and16['1 person household (%)_x']-single01and16['1 person household (%)_y']
single01and16.tail()

In [ ]:
single01and16.drop(['1 person household (%)_x','1 person household (%)_y'],axis=1, inplace=True)
single01and16.sort_values(['1 person household percentage difference between 2001 to 2016 (%)'], inplace=True ,ascending=True)
single01and16.reset_index(drop=True, inplace=True)
single01and16.tail()

In [ ]:
dm1=single01and16.copy()
dm2=f_marri2001.copy()

dm1=dm1[['Neighbourhood']]
dm2=dm2[['Neighbourhood']]

dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]
#dmiss=dm1[~dm1.apply(tuple,1).isin(dm2.apply(tuple,1))]

dmiss.sort_values(['Neighbourhood'], inplace=True ,ascending=True)
dmiss.reset_index(drop=True, inplace=True)
dmiss#.tail()

In [ ]:
dm1=single01and16.copy()
#dm2=single01and16.copy()
dm2=dclassMED.copy()

dm1=dm1[['Neighbourhood']]
dm2=dm2[['Neighbourhood']]

dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]
#dmiss=dm1[~dm1.apply(tuple,1).isin(dm2.apply(tuple,1))]

dmiss['1 person household percentage difference between 2001 to 2016 (%)']=[100]*len(dmiss)
dmiss.sort_values(['Neighbourhood'], inplace=True ,ascending=True)
dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

single01to16=single01and16.append(dmiss, ignore_index=True)

### HEAT MAP - % change

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisA.tail()

In [ ]:
NeisA.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisA,
                single01and16,
                 on='Neighbourhood')
#NeiAmerge = pd.merge(NeiAmerge,
#                dmiss,
#                 on='Neighbourhood')
#NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'Mill Woods Park', '1 person household percentage difference between 2001 to 2016 (%)'] = 0
#NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'University of Alberta Farm', '1 person household percentage difference between 2001 to 2016 (%)'] = 0
#NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'University of Alberta', '1 person household percentage difference between 2001 to 2016 (%)'] = 0
#NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'Place LaRue', '1 person household percentage difference between 2001 to 2016 (%)'] = 0
NeiAmerge.tail()

In [ ]:
NeisB.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiBmerge = pd.merge(NeisB,
                dmiss,
                 on='Neighbourhood')
NeiBmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09
#colormap = cm.linear.Paired_12.scale(-41, 81.4).to_step(12) 
colormap = cm.linear.PiYG_06.scale(-45, 45).to_step(6) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron')
colormap.caption = "% change"

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['1 person household percentage difference between 2001 to 2016 (%)']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Neighbourhood','1 person household percentage difference between 2001 to 2016 (%)'],
        aliases=['Neighbourhood','1 person household percentage difference between 2001 to 2016 (%)'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

from folium import plugins

def style_zero_function(feature):
    default_style = {
        'fillOpacity': 0.5,
        'color': 'grey',
        'weight': 0.1
    }
        
    default_style['fillPattern'] = plugins.pattern.StripePattern(angle=-45)
        
    return default_style

folium.GeoJson(
    NeiBmerge,
    style_function=style_zero_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['Neighbourhood'],
        aliases=['Neighbourhood'],
        localize=False
    ),
    ).add_to(edmA_map)

colormap.add_to(edmA_map)
edmA_map

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Percentage change in 1 person household (%)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#c51b7d;opacity:0.7;'></span><-30%</li>
    <li><span style='background:#e9a3c9;opacity:0.7;'></span>-30% to -15%</li>
    <li><span style='background:#fde0ef;opacity:0.7;'></span>-15% to 0%</li>
    <li><span style='background:#e6f5d0;opacity:0.7;'></span>0% to 15%</li>
    <li><span style='background:#a1d76a;opacity:0.7;'></span>15% to 30%</li>
    <li><span style='background:#4d9221;opacity:0.7;'></span>>30%</li>
    <li><span style='background:grey;opacity:0.7;'></span>new neighbourhood</li>


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edmA_map.get_root().add_child(macro)

edmA_map

## C6. Family with/without children
Reference：
 - https://public.tableau.com/profile/city.of.edmonton#!/vizhome/NeighbourhoodProfiles_FederalCensus2016/FamilyandLanguage
1. Download Tableau from the website
2. Access data by clicking 'data' from tableau and explore all into csv
3. upload to IBM_Cloud→Db2-vs→Manage→Open console→Load data (under GLN55437)

### Import data 2016

Import data from SQL SERVER

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from FAMILY_COUPLE"

#retrieve the query results into a pandas dataframe
f_couple = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
f_couple.head()

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from FAMILY_LONEPARENT"

#retrieve the query results into a pandas dataframe
f_lone = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
f_lone.head()

### Clean data 2016

In [ ]:
f_couple.rename(columns={'NAME':'Neighbourhood','Couple_Children_POP':'Family Number','Couple_Children_Group':'Children Group'}, inplace=True)
f_couple['Neighbourhood']=f_couple['Neighbourhood'].str.title()
f_couple.drop(['Nbhd_Num','Couple_Children_Percentage'],axis=1, inplace=True)
f_couple['Neighbourhood']=f_couple['Neighbourhood'].replace({'Mccauley':'McCauley',
                                                             'Mckernan':'McKernan',
                                                             'Mcleod':'McLeod',
                                                             'Mcqueen':'McQueen',
                                                             'Place Larue':'Place LaRue',
                                                             'Rapperswil':'Rapperswill',
                                                             "River'S Edge":"River's Edge",
                                                             'Westbrook Estate':'Westbrook Estates',
                                                             'Mcconachie Area':'McConachie Area',
                                                             'University Of Alberta Farm':'University of Alberta Farm',
                                                             'University Of Alberta':'University of Alberta',
                                                             'Glenridding Area':'Glenridding Heights',
                                                             'Macewan':'MacEwan',
                                                             'Central Mcdougall': 'Central McDougall'})
f_couple.sort_values(['Neighbourhood'], inplace=True ,ascending=True)


couple2016 = pd.merge(f_couple,
                      dclassMED[['Neighbourhood']],
                      on='Neighbourhood')
couple2016.tail()

In [ ]:
dm1=couple2016.copy()
dm2=dclassMED.copy()
dm1=dm1.groupby('Neighbourhood',axis=0)['Family Number'].sum().reset_index()
dm1=dm1[['Neighbourhood']]
dm2=dm2[['Neighbourhood']]
dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

In [ ]:
f_lone.rename(columns={'NAME':'Neighbourhood','Lone_Parent_Children_POP':'Family Number','Lone_Parent_Children':'Children Group'}, inplace=True)
f_lone['Neighbourhood']=f_lone['Neighbourhood'].str.title()
f_lone.drop(['Nbhd_Num','Lone_Parent_Children_Percentage'],axis=1, inplace=True)
f_lone['Neighbourhood']=f_lone['Neighbourhood'].replace({'Mccauley':'McCauley',
                                                             'Mckernan':'McKernan',
                                                             'Mcleod':'McLeod',
                                                             'Mcqueen':'McQueen',
                                                             'Place Larue':'Place LaRue',
                                                             'Rapperswil':'Rapperswill',
                                                             "River'S Edge":"River's Edge",
                                                             'Westbrook Estate':'Westbrook Estates',
                                                             'Mcconachie Area':'McConachie Area',
                                                             'University Of Alberta Farm':'University of Alberta Farm',
                                                             'University Of Alberta':'University of Alberta',
                                                             'Glenridding Area':'Glenridding Heights',
                                                             'Macewan':'MacEwan',
                                                             'Central Mcdougall': 'Central McDougall'})
f_lone.sort_values(['Neighbourhood'], inplace=True ,ascending=True)


lone2016 = pd.merge(f_lone,
                      dclassMED[['Neighbourhood']],
                      on='Neighbourhood')
lone2016.tail()

In [ ]:
fami2016=couple2016.copy()
fami2016= fami2016.append(lone2016, ignore_index=True)
fami2016.sort_values(['Neighbourhood'], inplace=True ,ascending=True)
fami2016.reset_index(drop=True, inplace=True)
fami2016.head(10)

In [ ]:
fami2016['Family Number']=fami2016['Family Number'].astype(float)
fami2016 = fami2016.groupby(['Children Group','Neighbourhood'],axis=0)['Family Number'].sum().reset_index()
fami2016.sort_values(['Neighbourhood'], inplace=True ,ascending=True)
fami2016.reset_index(drop=True, inplace=True)
fami2016.head(10)

In [ ]:
S_fami2016=fami2016.copy()
S_fami2016=S_fami2016.pivot(index='Neighbourhood',columns='Children Group',values='Family Number')
S_fami2016['Total']=S_fami2016.sum(axis=1)
S_fami2016['Without children (%)']=(S_fami2016['Without children']/S_fami2016['Total']*100).round(2)
S_fami2016['With children (%)']=((S_fami2016['One child']+S_fami2016['Two children']+S_fami2016['Three or more children'])/S_fami2016['Total']*100).round(2)
S_fami2016=S_fami2016.fillna(0)
S_fami2016.sort_values(['Without children (%)'], inplace=True ,ascending=True)
S_fami2016.head(10)

In [ ]:

a=S_fami2016.copy()
a=a['Without children (%)'].reset_index()
b=S_fami2016.copy()
b=b['With children (%)'].reset_index()
family2016 = pd.merge(a,
                b,
                 on='Neighbourhood')
#family2016.set_index('Neighbourhood', inplace=True)
family2016.tail()

### HEAT MAP - 2016

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisB = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisB.tail()

In [ ]:
NeisB.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisB,
                family2016[['Neighbourhood','Without children (%)']],
                 on='Neighbourhood')

#NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'Mill Woods Park', 'Without children (%)'] = 7.1 #reference: stat canada 2016 census
#NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'University of Alberta Farm', 'Without children (%)'] = 40.3
#NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'University of Alberta', 'Without children (%)'] = 88.6
#NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'Place LaRue', 'Without children (%)'] = 0
NeiAmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09
colormap = cm.linear.RdPu_08.scale(0, 60).to_step(6) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron')
colormap.caption = "Percentage of Census Families without children by neighbourhood"

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['Without children (%)']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Neighbourhood','Without children (%)'],
        aliases=['Neighbourhood','Without children (%)'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

colormap.add_to(edmA_map)

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Family without child (%)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#FEEBE2;opacity:0.7;'></span><10%</li>
    <li><span style='background:#FCC5C0;opacity:0.7;'></span>10% to 20%</li>
    <li><span style='background:#FA9FB5;opacity:0.7;'></span>20% to 30%</li>
    <li><span style='background:#F768A1;opacity:0.7;'></span>30% to 40%</li>
    <li><span style='background:#AE017E;opacity:0.7;'></span>40% to 50%</li>
    <li><span style='background:#7A0177;opacity:0.7;'></span>>50%</li>


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edmA_map.get_root().add_child(macro)

edmA_map

### Import data 2001

Import data from SQL SERVER    
Reference:
https://www.edmonton.ca/business_economy/documents/InfraPlan/FamilyHholds.pdf

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from FAMILY_SIZE2001"

#retrieve the query results into a pandas dataframe
f_children = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
f_children.head()

### Clean data 2001

In [ ]:
f_children.drop(f_children.head(2).index,inplace=True) # drop first n rows
f_children=f_children[['Neighbourhood','No_children','Total']]
f_children['No_children (%)']=f_children['No_children']/f_children['Total']*100
f_children.sort_values(['No_children (%)'], inplace=True ,ascending=True)
f_children.reset_index(drop=True, inplace=True)
f_children.tail()

### HEAT MAP - 2001

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisB = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisB.tail()

In [ ]:
NeisB.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisB,
                f_children[['Neighbourhood','No_children (%)']],
                 on='Neighbourhood')

NeiAmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09
colormap = cm.linear.RdPu_08.scale(0, 60).to_step(6) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron')
colormap.caption = "Percentage of 1 person household by neighbourhood in 2016"

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['No_children (%)']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Neighbourhood','No_children (%)'],
        aliases=['Neighbourhood','No_children (%)'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

colormap.add_to(edmA_map)
#edmA_map

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Family without child (%)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#FEEBE2;opacity:0.7;'></span><10%</li>
    <li><span style='background:#FCC5C0;opacity:0.7;'></span>10% to 20%</li>
    <li><span style='background:#FA9FB5;opacity:0.7;'></span>20% to 30%</li>
    <li><span style='background:#F768A1;opacity:0.7;'></span>30% to 40%</li>
    <li><span style='background:#AE017E;opacity:0.7;'></span>40% to 50%</li>
    <li><span style='background:#7A0177;opacity:0.7;'></span>>50%</li>


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edmA_map.get_root().add_child(macro)

edmA_map

### comparison 2001 and 2016

In [ ]:
f_children2001=f_children.copy()


f_children2001['Neighbourhood']=f_children2001['Neighbourhood'].replace({'Thorncliffe':'Thorncliff',
                                                                'Westbrook Estate':'Westbrook Estates',
                                                                'CFB Griesbach':'Griesbach',
                                                                 'Mcleod':'McLeod',
                                                                 'Millwoods Town Centre':'Mill Woods Town Centre',
                                                                 'Clareview Business Park':'Clareview Town Centre',
                                                                 'Place La Rue':'Place LaRue',
                                                                 'Eaux Claires - Belle Rive':'Eaux Claires',
                                                                 'Mayliewan - Ozerna':'Mayliewan',
                                                                 'Hollick Kenyon':'Hollick-Kenyon',
                                                                   'Rural West':'Rural West Big Lake',
                                                                   'Rural NE - Horse Hill':'Rural North East Horse Hill',
                                                                   'Rural NE - South Sturgeon':'Rural North East South Sturgeon'
                                                                })
f_children2001=f_children2001.append(f_children[f_children.Neighbourhood.str.contains("Eaux Claires - Belle Rive")], ignore_index=True)
f_children2001['Neighbourhood']=f_children2001['Neighbourhood'].replace({'Eaux Claires - Belle Rive':'Belle Rive'})
f_children2001=f_children2001.append(f_children[f_children.Neighbourhood.str.contains("Mayliewan - Ozerna")], ignore_index=True)
f_children2001['Neighbourhood']=f_children2001['Neighbourhood'].replace({'Mayliewan - Ozerna':'Ozerna'})
f_children2001=f_children2001.append(f_children[f_children.Neighbourhood.str.contains("Millwoods Town Centre")], ignore_index=True)
f_children2001['Neighbourhood']=f_children2001['Neighbourhood'].replace({'Millwoods Town Centre':'Mill Woods Park'})

child01and16 = pd.merge(family2016[['Neighbourhood','Without children (%)']],
                f_children2001[['Neighbourhood','No_children (%)']],
                 on='Neighbourhood')
child01and16['No children percentage difference between 2001 to 2016 (%)']=child01and16['Without children (%)']-child01and16['No_children (%)']
child01and16.tail()

In [ ]:
child01and16.drop(['Without children (%)','No_children (%)'],axis=1, inplace=True)
child01and16=child01and16.round(1)
child01and16.sort_values(['No children percentage difference between 2001 to 2016 (%)'], inplace=True ,ascending=True)
child01and16.reset_index(drop=True, inplace=True)
child01and16.tail()

In [ ]:
dm1=child01and16.copy()
dm2=f_children2001.copy()

dm1=dm1[['Neighbourhood']]
dm2=dm2[['Neighbourhood']]

dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]
#dmiss=dm1[~dm1.apply(tuple,1).isin(dm2.apply(tuple,1))]

dmiss.sort_values(['Neighbourhood'], inplace=True ,ascending=True)
dmiss.reset_index(drop=True, inplace=True)
dmiss#.tail()

In [ ]:
dm1=child01and16.copy()
#dm2=single01and16.copy()
dm2=dclassMED.copy()

dm1=dm1[['Neighbourhood']]
dm2=dm2[['Neighbourhood']]

dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]
#dmiss=dm1[~dm1.apply(tuple,1).isin(dm2.apply(tuple,1))]

dmiss['No children percentage difference between 2001 to 2016 (%)']=[100]*len(dmiss)
dmiss.sort_values(['Neighbourhood'], inplace=True ,ascending=True)
dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

child01and16=child01and16.append(dmiss, ignore_index=True)

### HEAT MAP - % change

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisB = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisB.tail()

In [ ]:
NeisB.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisB,
                child01and16,
                 on='Neighbourhood')

NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'Mill Woods Park', 'No children percentage difference between 2001 to 2016 (%)'] = 0
NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'University of Alberta Farm', 'No children percentage difference between 2001 to 2016 (%)'] = 0
NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'University of Alberta', 'No children percentage difference between 2001 to 2016 (%)'] = 0
NeiAmerge.loc[NeiAmerge['Neighbourhood'] == 'Place LaRue', 'No children percentage difference between 2001 to 2016 (%)'] = 0
NeiAmerge.tail()

In [ ]:
NeisB.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiBmerge = pd.merge(NeisB,
                dmiss,
                 on='Neighbourhood')
NeiBmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09
colormap = cm.linear.PiYG_06.scale(-30, 30).to_step(6) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron')
colormap.caption = "% change"

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['No children percentage difference between 2001 to 2016 (%)']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Neighbourhood','No children percentage difference between 2001 to 2016 (%)'],
        aliases=['Neighbourhood','No children percentage difference between 2001 to 2016 (%)'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

from folium import plugins

def style_zero_function(feature):
    default_style = {
        'fillOpacity': 0.5,
        'color': 'grey',
        'weight': 0.1
    }
        
    default_style['fillPattern'] = plugins.pattern.StripePattern(angle=-45)
        
    return default_style

folium.GeoJson(
    NeiBmerge,
    style_function=style_zero_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['Neighbourhood'],
        aliases=['Neighbourhood'],
        localize=False
    ),
    ).add_to(edmA_map)

colormap.add_to(edmA_map)
edmA_map

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Percentage change in family without child (%)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#c51b7d;opacity:0.7;'></span><-20%</li>
    <li><span style='background:#e9a3c9;opacity:0.7;'></span>-20% to -10%</li>
    <li><span style='background:#fde0ef;opacity:0.7;'></span>-10% to 0%</li>
    <li><span style='background:#e6f5d0;opacity:0.7;'></span>0% to 10%</li>
    <li><span style='background:#a1d76a;opacity:0.7;'></span>10% to 20%</li>
    <li><span style='background:#4d9221;opacity:0.7;'></span>>20%</li>
    <li><span style='background:grey;opacity:0.7;'></span>new neighbourhood</li>


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edmA_map.get_root().add_child(macro)

edmA_map

# **End**